Developed a plants seeds classifier using 5-fold cross validation on training data.
Picked the best model from it.

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
%matplotlib inline

In [ ]:
plants_data_dir = '../input/plant-seedlings-classification'

In [ ]:
os.listdir(plants_data_dir) 

In [ ]:
test_path = plants_data_dir+'/test/'
train_path = plants_data_dir+'/train/'

In [ ]:
categories = os.listdir(train_path)
categories

In [ ]:
len(categories)

In [ ]:
train_data = []
for category_id, category in enumerate(categories):
    for file in os.listdir(os.path.join(train_path, category)):
        train_data.append(['train/{}/{}'.format(category, file), category_id, category])
train = pd.DataFrame(train_data, columns=['imagefile', 'category_id', 'category'])
train.shape

In [ ]:
train.head(5)

# Find out the average dimensions of these images.

In [ ]:
# Test dataset image size
dim1 = []
dim2 = []
for image_filename in os.listdir(test_path):
    
    img = imread(test_path+'/'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

print(np.mean(dim1))
print(np.mean(dim2))

sns.jointplot(dim1,dim2)

In [ ]:
# Train dataset image size.

dim1_all = []
dim2_all = []

for plant_type in os.listdir(train_path):
  dim1 = []
  dim2 = []
  print(plant_type)
  for image_filename in os.listdir(train_path+'/'+plant_type):
      
      img = imread(train_path+'/'+plant_type+'/'+image_filename)
      d1,d2,colors = img.shape
      dim1.append(d1)
      dim2.append(d2)
      dim1_all.append(d1)
      dim2_all.append(d2)
  print(np.mean(dim1))
  print(np.mean(dim2))

sns.jointplot(dim1_all,dim2_all)
print(np.mean(dim1_all))
print(np.mean(dim2_all))

# Construct model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Create CNN model
def construct_model(activation, learning_rate, epochs, image_shape):

  # Including batch normalization
  model = Sequential()

  model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=image_shape, activation=activation)) 
  model.add(BatchNormalization())  
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #drop out to reduce overfitting
  model.add(Dropout(0.3))

  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation=activation))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation=activation))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation=activation))
  model.add(BatchNormalization())
 
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(256, activation=activation))

  model.add(Dense(12, activation='softmax'))  

  optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=learning_rate/epochs)
  model.compile(loss='categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
  model.summary()
  return model


# Early stopping 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

# Training the Model with k-fold split

In [ ]:
from sklearn import model_selection
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
SEED = 42

In [ ]:
def run_model(images, image_shape=(256,256,3), kfold_size=5, epochs=10, batch_size=16, activation='relu',learning_rate=0.001):

    print("Training started...")
    
    image_datagen_for_train = ImageDataGenerator(rotation_range=360, # rotate the image 20 degrees
                               width_shift_range=0.2, # Shift the pic width by a max of 0.3
                               height_shift_range=0.2, # Shift the pic height by a max of 0.3
                               rescale=1/255, # Rescale the image by normalzing it.
                               zoom_range=0.2, # Zoom in by 10% max
                               horizontal_flip=True, # Allow horizontal flipping
                               vertical_flip=True, # Allow vertical flipping
                              )
    image_datagen_for_vaild = ImageDataGenerator(rotation_range=360, # rotate the image 20 degrees
                               width_shift_range=0.2, # Shift the pic width by a max of 5%
                               height_shift_range=0.2, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               zoom_range=0.2, # Zoom in by 10% max
                               horizontal_flip=True, # Allow horizontal flipping
                               vertical_flip=True, # Allow vertical flipping
                              )
    # K-Fold selection.    
    kfold = model_selection.KFold(n_splits=kfold_size, shuffle = True, random_state=42)
    scores = []
    models = []
    iteration = 1

    target = images.category_id
    
    for train_index, valid_index in kfold.split(np.zeros(len(target)), target):

        print("***************************************************************")
        print("Iteration = ", iteration)

        # print("train index: " + str(train_index))
        # print("valid index: " + str(valid_index))
        train = images.loc[train_index]
        valid = images.loc[valid_index]
        
        train_generator = image_datagen_for_train.flow_from_dataframe(dataframe=train,
                                                  directory=plants_data_dir,
                                                  x_col="imagefile",
                                                  y_col="category",
                                                  batch_size=batch_size,
                                                  seed=SEED,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=image_shape[:2],
                                                  color_mode='rgb');

        valid_generator=image_datagen_for_vaild.flow_from_dataframe(dataframe=valid,
                                                  directory=plants_data_dir,
                                                  x_col="imagefile",
                                                  y_col="category",
                                                  batch_size=batch_size,
                                                  seed=SEED,
                                                  shuffle=False,
                                                  class_mode="categorical",
                                                  target_size=image_shape[:2],
                                                  color_mode='rgb');
        # Calcualte train/valid steps.
        STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
        STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

        model = construct_model(activation, learning_rate, epochs, image_shape);
        
        annealer = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, verbose=1, min_lr=1e-5)
        name='./model_'+str(iteration)+'.h5'
        checkpoint = ModelCheckpoint(name, verbose=1, save_best_only=True)

         # Trains the model on data generated batch-by-batch by a Python generator
        model.fit_generator(generator=train_generator,
                            validation_data = valid_generator,
                            steps_per_epoch=STEP_SIZE_TRAIN,
                            validation_steps=STEP_SIZE_VALID,
                            epochs=epochs,
                            verbose=1,
                            callbacks = [annealer, checkpoint])
        
        score = model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)
        print("Accuarcy %s: %.2f%%" % (model.metrics_names[1], score[1]*100))
        iteration = iteration + 1
        scores.append(score[1] * 100)
        models.append(model)
    
    print("***************************************************************")
    print('Training finished...')

    accuracy = np.mean(scores);
    std = np.std(scores);
    print("Accuracy: %.2f%% (+/- %.2f%%)" % (accuracy, std))

    # Save best model.
    best_model_index = np.argmax(scores, axis=0) 
    best_model= models[best_model_index]
    best_model.save('./best_model')

    return train_generator.class_indices

Developed a classifier using 5-fold cross validation on training data. picked the best model from it. used the best model to predict the test data.

Experiment with differnt image shapes such as,
*     image_shape=(66,66,3)
*     image_shape=(128,128,3)
*     image_shape=(256,256,3)
*     image_shape=(370,370,3) - Average image size. 

Best model found with image_shape=(66,66,3). 






In [ ]:
class_indices = run_model(
    images= train,
    image_shape=(66,66,3),
    kfold_size = 5,
    epochs = 80,
    batch_size =16,
    activation='relu',
    learning_rate=0.001
)
# Since training takes long time in keggle, train seperately and attached the logs below, 

In [ ]:
Training started...
***************************************************************
Iteration =  1
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         147584    
_________________________________________________________________
batch_normalization_4 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
=================================================================
Total params: 315,532
Trainable params: 314,700
Non-trainable params: 832
_________________________________________________________________
Epoch 1/80
236/237 [============================>.] - ETA: 0s - loss: 2.2350 - accuracy: 0.2410WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00001: val_loss improved from inf to 2.91473, saving model to ./model_1.h5
237/237 [==============================] - 39s 163ms/step - loss: 2.2335 - accuracy: 0.2413 - val_loss: 2.9147 - val_accuracy: 0.1462
Epoch 2/80
236/237 [============================>.] - ETA: 0s - loss: 1.7331 - accuracy: 0.4031WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00002: val_loss did not improve from 2.91473
237/237 [==============================] - 39s 163ms/step - loss: 1.7312 - accuracy: 0.4041 - val_loss: 3.2046 - val_accuracy: 0.1483
Epoch 3/80
236/237 [============================>.] - ETA: 0s - loss: 1.5483 - accuracy: 0.4658WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00003: val_loss improved from 2.91473 to 1.98084, saving model to ./model_1.h5
237/237 [==============================] - 39s 164ms/step - loss: 1.5472 - accuracy: 0.4664 - val_loss: 1.9808 - val_accuracy: 0.2987
Epoch 4/80
236/237 [============================>.] - ETA: 0s - loss: 1.3916 - accuracy: 0.5114WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00004: val_loss improved from 1.98084 to 1.50841, saving model to ./model_1.h5
237/237 [==============================] - 39s 163ms/step - loss: 1.3898 - accuracy: 0.5122 - val_loss: 1.5084 - val_accuracy: 0.4693
Epoch 5/80
236/237 [============================>.] - ETA: 0s - loss: 1.3011 - accuracy: 0.5456WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00005: val_loss improved from 1.50841 to 1.38014, saving model to ./model_1.h5
237/237 [==============================] - 39s 162ms/step - loss: 1.3004 - accuracy: 0.5457 - val_loss: 1.3801 - val_accuracy: 0.5201
Epoch 6/80
236/237 [============================>.] - ETA: 0s - loss: 1.2071 - accuracy: 0.5762WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00006: val_loss did not improve from 1.38014
237/237 [==============================] - 39s 163ms/step - loss: 1.2045 - accuracy: 0.5772 - val_loss: 1.8428 - val_accuracy: 0.3697
Epoch 7/80
236/237 [============================>.] - ETA: 0s - loss: 1.1037 - accuracy: 0.6165WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00007: val_loss did not improve from 1.38014
237/237 [==============================] - 39s 163ms/step - loss: 1.1031 - accuracy: 0.6160 - val_loss: 1.6849 - val_accuracy: 0.4439
Epoch 8/80
236/237 [============================>.] - ETA: 0s - loss: 1.0777 - accuracy: 0.6197WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00008: val_loss improved from 1.38014 to 1.25751, saving model to ./model_1.h5
237/237 [==============================] - 39s 163ms/step - loss: 1.0793 - accuracy: 0.6189 - val_loss: 1.2575 - val_accuracy: 0.5540
Epoch 9/80
236/237 [============================>.] - ETA: 0s - loss: 1.0119 - accuracy: 0.6576WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00009: val_loss did not improve from 1.25751
237/237 [==============================] - 38s 162ms/step - loss: 1.0125 - accuracy: 0.6575 - val_loss: 2.6951 - val_accuracy: 0.2818
Epoch 10/80
236/237 [============================>.] - ETA: 0s - loss: 0.9561 - accuracy: 0.6600WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00010: val_loss did not improve from 1.25751
237/237 [==============================] - 38s 161ms/step - loss: 0.9566 - accuracy: 0.6599 - val_loss: 1.9180 - val_accuracy: 0.4311
Epoch 11/80
236/237 [============================>.] - ETA: 0s - loss: 0.9021 - accuracy: 0.6844WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00011: val_loss improved from 1.25751 to 1.13400, saving model to ./model_1.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.9030 - accuracy: 0.6839 - val_loss: 1.1340 - val_accuracy: 0.5975
Epoch 12/80
236/237 [============================>.] - ETA: 0s - loss: 0.8518 - accuracy: 0.6975WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00012: val_loss did not improve from 1.13400
237/237 [==============================] - 38s 159ms/step - loss: 0.8513 - accuracy: 0.6974 - val_loss: 1.8406 - val_accuracy: 0.3835
Epoch 13/80
236/237 [============================>.] - ETA: 0s - loss: 0.8140 - accuracy: 0.7150WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00013: val_loss did not improve from 1.13400
237/237 [==============================] - 38s 158ms/step - loss: 0.8160 - accuracy: 0.7146 - val_loss: 1.9227 - val_accuracy: 0.3919
Epoch 14/80
236/237 [============================>.] - ETA: 0s - loss: 0.8256 - accuracy: 0.7142WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00014: val_loss did not improve from 1.13400
237/237 [==============================] - 38s 158ms/step - loss: 0.8246 - accuracy: 0.7143 - val_loss: 1.4833 - val_accuracy: 0.5064
Epoch 15/80
236/237 [============================>.] - ETA: 0s - loss: 0.7999 - accuracy: 0.7208WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00015: val_loss improved from 1.13400 to 1.04062, saving model to ./model_1.h5
237/237 [==============================] - 38s 161ms/step - loss: 0.7994 - accuracy: 0.7212 - val_loss: 1.0406 - val_accuracy: 0.6589
Epoch 16/80
236/237 [============================>.] - ETA: 0s - loss: 0.7750 - accuracy: 0.7343WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00016: val_loss did not improve from 1.04062
237/237 [==============================] - 38s 159ms/step - loss: 0.7766 - accuracy: 0.7336 - val_loss: 1.6316 - val_accuracy: 0.4915
Epoch 17/80
236/237 [============================>.] - ETA: 0s - loss: 0.7250 - accuracy: 0.7375WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00017: val_loss improved from 1.04062 to 0.85382, saving model to ./model_1.h5
237/237 [==============================] - 38s 160ms/step - loss: 0.7240 - accuracy: 0.7381 - val_loss: 0.8538 - val_accuracy: 0.7013
Epoch 18/80
236/237 [============================>.] - ETA: 0s - loss: 0.7421 - accuracy: 0.7460WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00018: val_loss did not improve from 0.85382
237/237 [==============================] - 38s 160ms/step - loss: 0.7417 - accuracy: 0.7460 - val_loss: 1.4923 - val_accuracy: 0.5222
Epoch 19/80
236/237 [============================>.] - ETA: 0s - loss: 0.7030 - accuracy: 0.7487WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00019: val_loss did not improve from 0.85382
237/237 [==============================] - 37s 158ms/step - loss: 0.7046 - accuracy: 0.7482 - val_loss: 0.9128 - val_accuracy: 0.6896
Epoch 20/80
236/237 [============================>.] - ETA: 0s - loss: 0.6768 - accuracy: 0.7643WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00020: val_loss did not improve from 0.85382
237/237 [==============================] - 37s 158ms/step - loss: 0.6764 - accuracy: 0.7640 - val_loss: 1.1401 - val_accuracy: 0.6038
Epoch 21/80
236/237 [============================>.] - ETA: 0s - loss: 0.6514 - accuracy: 0.7673WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00021: val_loss improved from 0.85382 to 0.81976, saving model to ./model_1.h5
237/237 [==============================] - 37s 158ms/step - loss: 0.6500 - accuracy: 0.7680 - val_loss: 0.8198 - val_accuracy: 0.7415
Epoch 22/80
236/237 [============================>.] - ETA: 0s - loss: 0.6576 - accuracy: 0.7667WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00022: val_loss did not improve from 0.81976
237/237 [==============================] - 38s 159ms/step - loss: 0.6585 - accuracy: 0.7664 - val_loss: 1.2421 - val_accuracy: 0.5996
Epoch 23/80
236/237 [============================>.] - ETA: 0s - loss: 0.6355 - accuracy: 0.7760WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00023: val_loss improved from 0.81976 to 0.78092, saving model to ./model_1.h5
237/237 [==============================] - 38s 158ms/step - loss: 0.6368 - accuracy: 0.7756 - val_loss: 0.7809 - val_accuracy: 0.7373
Epoch 24/80
236/237 [============================>.] - ETA: 0s - loss: 0.6187 - accuracy: 0.7710WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00024: val_loss improved from 0.78092 to 0.75510, saving model to ./model_1.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.6174 - accuracy: 0.7715 - val_loss: 0.7551 - val_accuracy: 0.7383
Epoch 25/80
236/237 [============================>.] - ETA: 0s - loss: 0.6275 - accuracy: 0.7787WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00025: val_loss did not improve from 0.75510
237/237 [==============================] - 38s 159ms/step - loss: 0.6301 - accuracy: 0.7783 - val_loss: 0.8890 - val_accuracy: 0.6949
Epoch 26/80
236/237 [============================>.] - ETA: 0s - loss: 0.5989 - accuracy: 0.7880WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00026: val_loss did not improve from 0.75510
237/237 [==============================] - 38s 160ms/step - loss: 0.5990 - accuracy: 0.7881 - val_loss: 0.9367 - val_accuracy: 0.6886
Epoch 27/80
236/237 [============================>.] - ETA: 0s - loss: 0.5633 - accuracy: 0.8012WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00027: val_loss improved from 0.75510 to 0.59587, saving model to ./model_1.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.5644 - accuracy: 0.8007 - val_loss: 0.5959 - val_accuracy: 0.8051
Epoch 28/80
236/237 [============================>.] - ETA: 0s - loss: 0.5709 - accuracy: 0.7873WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00028: val_loss did not improve from 0.59587
237/237 [==============================] - 38s 161ms/step - loss: 0.5720 - accuracy: 0.7872 - val_loss: 0.7453 - val_accuracy: 0.7564
Epoch 29/80
236/237 [============================>.] - ETA: 0s - loss: 0.5805 - accuracy: 0.7923WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00029: val_loss did not improve from 0.59587
237/237 [==============================] - 38s 161ms/step - loss: 0.5794 - accuracy: 0.7929 - val_loss: 1.4738 - val_accuracy: 0.5328
Epoch 30/80
236/237 [============================>.] - ETA: 0s - loss: 0.5513 - accuracy: 0.8076WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00030: val_loss did not improve from 0.59587
237/237 [==============================] - 38s 159ms/step - loss: 0.5506 - accuracy: 0.8079 - val_loss: 0.8393 - val_accuracy: 0.6706
Epoch 31/80
236/237 [============================>.] - ETA: 0s - loss: 0.5740 - accuracy: 0.8023WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00031: val_loss did not improve from 0.59587
237/237 [==============================] - 38s 159ms/step - loss: 0.5739 - accuracy: 0.8026 - val_loss: 0.8814 - val_accuracy: 0.6833
Epoch 32/80
236/237 [============================>.] - ETA: 0s - loss: 0.5255 - accuracy: 0.8089WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00032: val_loss improved from 0.59587 to 0.56372, saving model to ./model_1.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.5240 - accuracy: 0.8097 - val_loss: 0.5637 - val_accuracy: 0.7998
Epoch 33/80
236/237 [============================>.] - ETA: 0s - loss: 0.5421 - accuracy: 0.8063WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00033: val_loss did not improve from 0.56372
237/237 [==============================] - 37s 156ms/step - loss: 0.5434 - accuracy: 0.8058 - val_loss: 1.0235 - val_accuracy: 0.6737
Epoch 34/80
236/237 [============================>.] - ETA: 0s - loss: 0.5138 - accuracy: 0.8120WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00034: val_loss did not improve from 0.56372
237/237 [==============================] - 37s 158ms/step - loss: 0.5124 - accuracy: 0.8125 - val_loss: 0.9021 - val_accuracy: 0.7076
Epoch 35/80
236/237 [============================>.] - ETA: 0s - loss: 0.5339 - accuracy: 0.8080WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00035: val_loss did not improve from 0.56372
237/237 [==============================] - 37s 155ms/step - loss: 0.5361 - accuracy: 0.8075 - val_loss: 0.5792 - val_accuracy: 0.7998
Epoch 36/80
236/237 [============================>.] - ETA: 0s - loss: 0.5369 - accuracy: 0.8073WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00036: val_loss did not improve from 0.56372
237/237 [==============================] - 38s 159ms/step - loss: 0.5367 - accuracy: 0.8073 - val_loss: 1.0742 - val_accuracy: 0.6568
Epoch 37/80
236/237 [============================>.] - ETA: 0s - loss: 0.4877 - accuracy: 0.8280WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00037: val_loss improved from 0.56372 to 0.53267, saving model to ./model_1.h5
237/237 [==============================] - 37s 158ms/step - loss: 0.4865 - accuracy: 0.8285 - val_loss: 0.5327 - val_accuracy: 0.8294
Epoch 38/80
236/237 [============================>.] - ETA: 0s - loss: 0.5083 - accuracy: 0.8140WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00038: val_loss did not improve from 0.53267
237/237 [==============================] - 38s 159ms/step - loss: 0.5098 - accuracy: 0.8137 - val_loss: 1.6034 - val_accuracy: 0.5667
Epoch 39/80
236/237 [============================>.] - ETA: 0s - loss: 0.4835 - accuracy: 0.8193WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00039: val_loss did not improve from 0.53267
237/237 [==============================] - 38s 158ms/step - loss: 0.4842 - accuracy: 0.8195 - val_loss: 0.9067 - val_accuracy: 0.7161
Epoch 40/80
236/237 [============================>.] - ETA: 0s - loss: 0.5005 - accuracy: 0.8270WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00040: val_loss did not improve from 0.53267
237/237 [==============================] - 38s 160ms/step - loss: 0.5018 - accuracy: 0.8266 - val_loss: 1.4562 - val_accuracy: 0.5900
Epoch 41/80
236/237 [============================>.] - ETA: 0s - loss: 0.4874 - accuracy: 0.8190WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00041: val_loss did not improve from 0.53267
237/237 [==============================] - 38s 159ms/step - loss: 0.4880 - accuracy: 0.8192 - val_loss: 1.5075 - val_accuracy: 0.6049
Epoch 42/80
236/237 [============================>.] - ETA: 0s - loss: 0.4721 - accuracy: 0.8249WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00042: val_loss did not improve from 0.53267
237/237 [==============================] - 38s 159ms/step - loss: 0.4720 - accuracy: 0.8252 - val_loss: 0.8003 - val_accuracy: 0.7193
Epoch 43/80
236/237 [============================>.] - ETA: 0s - loss: 0.4509 - accuracy: 0.8370WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00043: val_loss did not improve from 0.53267
237/237 [==============================] - 37s 156ms/step - loss: 0.4507 - accuracy: 0.8369 - val_loss: 0.7815 - val_accuracy: 0.7447
Epoch 44/80
236/237 [============================>.] - ETA: 0s - loss: 0.5042 - accuracy: 0.8110WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00044: val_loss did not improve from 0.53267
237/237 [==============================] - 37s 157ms/step - loss: 0.5032 - accuracy: 0.8116 - val_loss: 0.8828 - val_accuracy: 0.7288
Epoch 45/80
236/237 [============================>.] - ETA: 0s - loss: 0.4677 - accuracy: 0.8240WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00045: val_loss improved from 0.53267 to 0.41410, saving model to ./model_1.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.4680 - accuracy: 0.8243 - val_loss: 0.4141 - val_accuracy: 0.8633
Epoch 46/80
236/237 [============================>.] - ETA: 0s - loss: 0.4500 - accuracy: 0.8349WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00046: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 158ms/step - loss: 0.4512 - accuracy: 0.8348 - val_loss: 0.7462 - val_accuracy: 0.7426
Epoch 47/80
236/237 [============================>.] - ETA: 0s - loss: 0.4525 - accuracy: 0.8386WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00047: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4512 - accuracy: 0.8393 - val_loss: 0.5607 - val_accuracy: 0.8146
Epoch 48/80
236/237 [============================>.] - ETA: 0s - loss: 0.4450 - accuracy: 0.8381WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00048: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4458 - accuracy: 0.8377 - val_loss: 0.6937 - val_accuracy: 0.7574
Epoch 49/80
236/237 [============================>.] - ETA: 0s - loss: 0.4348 - accuracy: 0.8432WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00049: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 161ms/step - loss: 0.4349 - accuracy: 0.8428 - val_loss: 0.5377 - val_accuracy: 0.8083
Epoch 50/80
236/237 [============================>.] - ETA: 0s - loss: 0.4233 - accuracy: 0.8444WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00050: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4250 - accuracy: 0.8435 - val_loss: 0.5268 - val_accuracy: 0.8220
Epoch 51/80
236/237 [============================>.] - ETA: 0s - loss: 0.4404 - accuracy: 0.8392WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00051: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4409 - accuracy: 0.8393 - val_loss: 0.5111 - val_accuracy: 0.8210
Epoch 52/80
236/237 [============================>.] - ETA: 0s - loss: 0.4292 - accuracy: 0.8402WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00052: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.4308 - accuracy: 0.8396 - val_loss: 0.8988 - val_accuracy: 0.6886
Epoch 53/80
236/237 [============================>.] - ETA: 0s - loss: 0.4197 - accuracy: 0.8477WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00053: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.4191 - accuracy: 0.8478 - val_loss: 1.3367 - val_accuracy: 0.5890
Epoch 54/80
236/237 [============================>.] - ETA: 0s - loss: 0.4483 - accuracy: 0.8356WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00054: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.4473 - accuracy: 0.8363 - val_loss: 0.5338 - val_accuracy: 0.8072
Epoch 55/80
236/237 [============================>.] - ETA: 0s - loss: 0.4073 - accuracy: 0.8501WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00055: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4070 - accuracy: 0.8502 - val_loss: 0.6469 - val_accuracy: 0.7956
Epoch 56/80
236/237 [============================>.] - ETA: 0s - loss: 0.4166 - accuracy: 0.8524WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00056: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.4158 - accuracy: 0.8528 - val_loss: 0.6252 - val_accuracy: 0.7828
Epoch 57/80
236/237 [============================>.] - ETA: 0s - loss: 0.4314 - accuracy: 0.8400WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00057: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.4330 - accuracy: 0.8399 - val_loss: 0.4367 - val_accuracy: 0.8538
Epoch 58/80
236/237 [============================>.] - ETA: 0s - loss: 0.4199 - accuracy: 0.8442WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00058: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.4211 - accuracy: 0.8438 - val_loss: 1.9583 - val_accuracy: 0.4767
Epoch 59/80
236/237 [============================>.] - ETA: 0s - loss: 0.4195 - accuracy: 0.8429WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00059: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 158ms/step - loss: 0.4205 - accuracy: 0.8420 - val_loss: 0.5171 - val_accuracy: 0.8284
Epoch 60/80
236/237 [============================>.] - ETA: 0s - loss: 0.3999 - accuracy: 0.8543WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00060: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 154ms/step - loss: 0.3995 - accuracy: 0.8541 - val_loss: 0.8797 - val_accuracy: 0.7394
Epoch 61/80
236/237 [============================>.] - ETA: 0s - loss: 0.4145 - accuracy: 0.8501WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00061: val_loss did not improve from 0.41410
237/237 [==============================] - 39s 163ms/step - loss: 0.4149 - accuracy: 0.8502 - val_loss: 0.4957 - val_accuracy: 0.8390
Epoch 62/80
236/237 [============================>.] - ETA: 0s - loss: 0.4185 - accuracy: 0.8497WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00062: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.4185 - accuracy: 0.8501 - val_loss: 0.6729 - val_accuracy: 0.7850
Epoch 63/80
236/237 [============================>.] - ETA: 0s - loss: 0.3849 - accuracy: 0.8564WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00063: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 158ms/step - loss: 0.3846 - accuracy: 0.8565 - val_loss: 1.5904 - val_accuracy: 0.5964
Epoch 64/80
236/237 [============================>.] - ETA: 0s - loss: 0.4025 - accuracy: 0.8501WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00064: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.4032 - accuracy: 0.8496 - val_loss: 0.5347 - val_accuracy: 0.8178
Epoch 65/80
236/237 [============================>.] - ETA: 0s - loss: 0.3815 - accuracy: 0.8551WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00065: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 162ms/step - loss: 0.3818 - accuracy: 0.8549 - val_loss: 0.5034 - val_accuracy: 0.8252
Epoch 66/80
236/237 [============================>.] - ETA: 0s - loss: 0.3909 - accuracy: 0.8528WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00066: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 155ms/step - loss: 0.3910 - accuracy: 0.8528 - val_loss: 0.4409 - val_accuracy: 0.8528
Epoch 67/80
236/237 [============================>.] - ETA: 0s - loss: 0.3830 - accuracy: 0.8617WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00067: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 162ms/step - loss: 0.3824 - accuracy: 0.8621 - val_loss: 0.5660 - val_accuracy: 0.8326
Epoch 68/80
236/237 [============================>.] - ETA: 0s - loss: 0.3829 - accuracy: 0.8607WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00068: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.3821 - accuracy: 0.8607 - val_loss: 0.6431 - val_accuracy: 0.7860
Epoch 69/80
236/237 [============================>.] - ETA: 0s - loss: 0.3637 - accuracy: 0.8614WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00069: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 157ms/step - loss: 0.3625 - accuracy: 0.8620 - val_loss: 1.0282 - val_accuracy: 0.6737
Epoch 70/80
236/237 [============================>.] - ETA: 0s - loss: 0.3894 - accuracy: 0.8512WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00070: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 162ms/step - loss: 0.3891 - accuracy: 0.8515 - val_loss: 0.4901 - val_accuracy: 0.8400
Epoch 71/80
236/237 [============================>.] - ETA: 0s - loss: 0.3808 - accuracy: 0.8593WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00071: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.3810 - accuracy: 0.8596 - val_loss: 1.4645 - val_accuracy: 0.6144
Epoch 72/80
236/237 [============================>.] - ETA: 0s - loss: 0.3832 - accuracy: 0.8585WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00072: val_loss did not improve from 0.41410
237/237 [==============================] - 36s 154ms/step - loss: 0.3837 - accuracy: 0.8584 - val_loss: 0.9761 - val_accuracy: 0.6494
Epoch 73/80
236/237 [============================>.] - ETA: 0s - loss: 0.3880 - accuracy: 0.8541WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00073: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 158ms/step - loss: 0.3886 - accuracy: 0.8539 - val_loss: 0.4772 - val_accuracy: 0.8453
Epoch 74/80
236/237 [============================>.] - ETA: 0s - loss: 0.3560 - accuracy: 0.8673WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00074: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.3555 - accuracy: 0.8671 - val_loss: 1.3332 - val_accuracy: 0.6144
Epoch 75/80
236/237 [============================>.] - ETA: 0s - loss: 0.3672 - accuracy: 0.8630WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00075: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 156ms/step - loss: 0.3667 - accuracy: 0.8633 - val_loss: 1.0563 - val_accuracy: 0.6642
Epoch 76/80
236/237 [============================>.] - ETA: 0s - loss: 0.3605 - accuracy: 0.8652WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00076: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.3607 - accuracy: 0.8650 - val_loss: 0.7791 - val_accuracy: 0.7479
Epoch 77/80
236/237 [============================>.] - ETA: 0s - loss: 0.3692 - accuracy: 0.8646WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00077: val_loss did not improve from 0.41410
237/237 [==============================] - 37s 158ms/step - loss: 0.3692 - accuracy: 0.8647 - val_loss: 0.5458 - val_accuracy: 0.8008
Epoch 78/80
236/237 [============================>.] - ETA: 0s - loss: 0.3380 - accuracy: 0.8747WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00078: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 161ms/step - loss: 0.3380 - accuracy: 0.8750 - val_loss: 1.1043 - val_accuracy: 0.6981
Epoch 79/80
236/237 [============================>.] - ETA: 0s - loss: 0.3651 - accuracy: 0.8636WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00079: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 159ms/step - loss: 0.3654 - accuracy: 0.8634 - val_loss: 0.4333 - val_accuracy: 0.8591
Epoch 80/80
236/237 [============================>.] - ETA: 0s - loss: 0.3571 - accuracy: 0.8702WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00080: val_loss did not improve from 0.41410
237/237 [==============================] - 38s 160ms/step - loss: 0.3578 - accuracy: 0.8695 - val_loss: 0.7055 - val_accuracy: 0.7691
Accuarcy accuracy: 75.53%
***************************************************************
Iteration =  2
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_5 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_7 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
batch_normalization_8 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 2, 128)         147584    
_________________________________________________________________
batch_normalization_9 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3084      
=================================================================
Total params: 315,532
Trainable params: 314,700
Non-trainable params: 832
_________________________________________________________________
Epoch 1/80
236/237 [============================>.] - ETA: 0s - loss: 2.4062 - accuracy: 0.1988WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00001: val_loss improved from inf to 2.65796, saving model to ./model_2.h5
237/237 [==============================] - 39s 164ms/step - loss: 2.4042 - accuracy: 0.1998 - val_loss: 2.6580 - val_accuracy: 0.1292
Epoch 2/80
236/237 [============================>.] - ETA: 0s - loss: 1.9000 - accuracy: 0.3416WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00002: val_loss improved from 2.65796 to 2.36778, saving model to ./model_2.h5
237/237 [==============================] - 39s 163ms/step - loss: 1.8986 - accuracy: 0.3420 - val_loss: 2.3678 - val_accuracy: 0.2002
Epoch 3/80
236/237 [============================>.] - ETA: 0s - loss: 1.6530 - accuracy: 0.4262WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00003: val_loss improved from 2.36778 to 2.15998, saving model to ./model_2.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.6528 - accuracy: 0.4265 - val_loss: 2.1600 - val_accuracy: 0.3008
Epoch 4/80
236/237 [============================>.] - ETA: 0s - loss: 1.4877 - accuracy: 0.4854WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00004: val_loss improved from 2.15998 to 2.02400, saving model to ./model_2.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.4857 - accuracy: 0.4863 - val_loss: 2.0240 - val_accuracy: 0.3125
Epoch 5/80
236/237 [============================>.] - ETA: 0s - loss: 1.3697 - accuracy: 0.5231WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00005: val_loss did not improve from 2.02400
237/237 [==============================] - 38s 162ms/step - loss: 1.3699 - accuracy: 0.5230 - val_loss: 2.1336 - val_accuracy: 0.3019
Epoch 6/80
236/237 [============================>.] - ETA: 0s - loss: 1.2855 - accuracy: 0.5663WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00006: val_loss improved from 2.02400 to 1.37686, saving model to ./model_2.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.2841 - accuracy: 0.5674 - val_loss: 1.3769 - val_accuracy: 0.5159
Epoch 7/80
236/237 [============================>.] - ETA: 0s - loss: 1.1843 - accuracy: 0.5985WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00007: val_loss improved from 1.37686 to 1.24399, saving model to ./model_2.h5
237/237 [==============================] - 39s 163ms/step - loss: 1.1847 - accuracy: 0.5980 - val_loss: 1.2440 - val_accuracy: 0.5487
Epoch 8/80
236/237 [============================>.] - ETA: 0s - loss: 1.0895 - accuracy: 0.6255WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00008: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 161ms/step - loss: 1.0898 - accuracy: 0.6253 - val_loss: 1.9653 - val_accuracy: 0.3422
Epoch 9/80
236/237 [============================>.] - ETA: 0s - loss: 1.0424 - accuracy: 0.6462WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00009: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 160ms/step - loss: 1.0410 - accuracy: 0.6464 - val_loss: 1.6301 - val_accuracy: 0.4269
Epoch 10/80
236/237 [============================>.] - ETA: 0s - loss: 0.9859 - accuracy: 0.6539WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00010: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 159ms/step - loss: 0.9861 - accuracy: 0.6543 - val_loss: 1.3061 - val_accuracy: 0.5519
Epoch 11/80
236/237 [============================>.] - ETA: 0s - loss: 0.9330 - accuracy: 0.6730WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00011: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 160ms/step - loss: 0.9320 - accuracy: 0.6734 - val_loss: 1.5384 - val_accuracy: 0.4820
Epoch 12/80
236/237 [============================>.] - ETA: 0s - loss: 0.8916 - accuracy: 0.6913WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00012: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 159ms/step - loss: 0.8913 - accuracy: 0.6911 - val_loss: 2.5254 - val_accuracy: 0.3496
Epoch 13/80
236/237 [============================>.] - ETA: 0s - loss: 0.8712 - accuracy: 0.6969WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00013: val_loss did not improve from 1.24399
237/237 [==============================] - 38s 160ms/step - loss: 0.8738 - accuracy: 0.6956 - val_loss: 1.2916 - val_accuracy: 0.5551
Epoch 14/80
236/237 [============================>.] - ETA: 0s - loss: 0.8441 - accuracy: 0.7086WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00014: val_loss improved from 1.24399 to 0.82548, saving model to ./model_2.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.8445 - accuracy: 0.7085 - val_loss: 0.8255 - val_accuracy: 0.6939
Epoch 15/80
236/237 [============================>.] - ETA: 0s - loss: 0.8132 - accuracy: 0.7168WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00015: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 157ms/step - loss: 0.8119 - accuracy: 0.7175 - val_loss: 1.1079 - val_accuracy: 0.5975
Epoch 16/80
236/237 [============================>.] - ETA: 0s - loss: 0.7851 - accuracy: 0.7258WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00016: val_loss did not improve from 0.82548
237/237 [==============================] - 38s 159ms/step - loss: 0.7856 - accuracy: 0.7257 - val_loss: 1.9305 - val_accuracy: 0.4492
Epoch 17/80
236/237 [============================>.] - ETA: 0s - loss: 0.7658 - accuracy: 0.7322WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00017: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 157ms/step - loss: 0.7646 - accuracy: 0.7326 - val_loss: 1.7225 - val_accuracy: 0.4852
Epoch 18/80
236/237 [============================>.] - ETA: 0s - loss: 0.7362 - accuracy: 0.7473WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00018: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 157ms/step - loss: 0.7355 - accuracy: 0.7479 - val_loss: 3.5683 - val_accuracy: 0.2924
Epoch 19/80
236/237 [============================>.] - ETA: 0s - loss: 0.7029 - accuracy: 0.7535WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00019: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 156ms/step - loss: 0.7016 - accuracy: 0.7542 - val_loss: 1.1022 - val_accuracy: 0.5710
Epoch 20/80
236/237 [============================>.] - ETA: 0s - loss: 0.7025 - accuracy: 0.7524WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00020: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 157ms/step - loss: 0.7037 - accuracy: 0.7521 - val_loss: 0.9970 - val_accuracy: 0.6727
Epoch 21/80
236/237 [============================>.] - ETA: 0s - loss: 0.6835 - accuracy: 0.7566WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00021: val_loss did not improve from 0.82548
237/237 [==============================] - 38s 159ms/step - loss: 0.6834 - accuracy: 0.7563 - val_loss: 1.0564 - val_accuracy: 0.6261
Epoch 22/80
236/237 [============================>.] - ETA: 0s - loss: 0.6513 - accuracy: 0.7731WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00022: val_loss did not improve from 0.82548
237/237 [==============================] - 37s 157ms/step - loss: 0.6519 - accuracy: 0.7730 - val_loss: 0.9209 - val_accuracy: 0.6769
Epoch 23/80
236/237 [============================>.] - ETA: 0s - loss: 0.6482 - accuracy: 0.7739WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00023: val_loss did not improve from 0.82548
237/237 [==============================] - 38s 159ms/step - loss: 0.6480 - accuracy: 0.7738 - val_loss: 1.1050 - val_accuracy: 0.6144
Epoch 24/80
236/237 [============================>.] - ETA: 0s - loss: 0.6449 - accuracy: 0.7821WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00024: val_loss improved from 0.82548 to 0.78801, saving model to ./model_2.h5
237/237 [==============================] - 37s 158ms/step - loss: 0.6454 - accuracy: 0.7820 - val_loss: 0.7880 - val_accuracy: 0.7140
Epoch 25/80
236/237 [============================>.] - ETA: 0s - loss: 0.6291 - accuracy: 0.7755WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00025: val_loss improved from 0.78801 to 0.76394, saving model to ./model_2.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.6319 - accuracy: 0.7746 - val_loss: 0.7639 - val_accuracy: 0.7341
Epoch 26/80
236/237 [============================>.] - ETA: 0s - loss: 0.5974 - accuracy: 0.7921WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00026: val_loss improved from 0.76394 to 0.56233, saving model to ./model_2.h5
237/237 [==============================] - 37s 158ms/step - loss: 0.5967 - accuracy: 0.7922 - val_loss: 0.5623 - val_accuracy: 0.8072
Epoch 27/80
236/237 [============================>.] - ETA: 0s - loss: 0.5820 - accuracy: 0.7934WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00027: val_loss did not improve from 0.56233
237/237 [==============================] - 38s 160ms/step - loss: 0.5829 - accuracy: 0.7929 - val_loss: 1.1401 - val_accuracy: 0.6049
Epoch 28/80
236/237 [============================>.] - ETA: 0s - loss: 0.5934 - accuracy: 0.7932WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00028: val_loss did not improve from 0.56233
237/237 [==============================] - 38s 159ms/step - loss: 0.5926 - accuracy: 0.7935 - val_loss: 0.9005 - val_accuracy: 0.6822
Epoch 29/80
236/237 [============================>.] - ETA: 0s - loss: 0.5795 - accuracy: 0.8027WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00029: val_loss did not improve from 0.56233
237/237 [==============================] - 37s 154ms/step - loss: 0.5802 - accuracy: 0.8019 - val_loss: 0.8249 - val_accuracy: 0.6949
Epoch 30/80
236/237 [============================>.] - ETA: 0s - loss: 0.5660 - accuracy: 0.7935WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00030: val_loss did not improve from 0.56233
237/237 [==============================] - 38s 160ms/step - loss: 0.5649 - accuracy: 0.7939 - val_loss: 0.5932 - val_accuracy: 0.7913
Epoch 31/80
236/237 [============================>.] - ETA: 0s - loss: 0.5449 - accuracy: 0.8032WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00031: val_loss did not improve from 0.56233
237/237 [==============================] - 38s 159ms/step - loss: 0.5449 - accuracy: 0.8033 - val_loss: 1.5130 - val_accuracy: 0.5350
Epoch 32/80
236/237 [============================>.] - ETA: 0s - loss: 0.5508 - accuracy: 0.8035WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00032: val_loss did not improve from 0.56233
237/237 [==============================] - 37s 155ms/step - loss: 0.5525 - accuracy: 0.8027 - val_loss: 2.3978 - val_accuracy: 0.4163
Epoch 33/80
236/237 [============================>.] - ETA: 0s - loss: 0.5456 - accuracy: 0.8110WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00033: val_loss did not improve from 0.56233
237/237 [==============================] - 38s 159ms/step - loss: 0.5492 - accuracy: 0.8100 - val_loss: 0.5634 - val_accuracy: 0.7956
Epoch 34/80
236/237 [============================>.] - ETA: 0s - loss: 0.5278 - accuracy: 0.8118WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00034: val_loss did not improve from 0.56233
237/237 [==============================] - 37s 155ms/step - loss: 0.5282 - accuracy: 0.8118 - val_loss: 2.9130 - val_accuracy: 0.3655
Epoch 35/80
236/237 [============================>.] - ETA: 0s - loss: 0.5372 - accuracy: 0.8151WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00035: val_loss improved from 0.56233 to 0.54915, saving model to ./model_2.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.5364 - accuracy: 0.8151 - val_loss: 0.5492 - val_accuracy: 0.8040
Epoch 36/80
236/237 [============================>.] - ETA: 0s - loss: 0.5104 - accuracy: 0.8152WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00036: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 158ms/step - loss: 0.5127 - accuracy: 0.8141 - val_loss: 1.5413 - val_accuracy: 0.5784
Epoch 37/80
236/237 [============================>.] - ETA: 0s - loss: 0.5148 - accuracy: 0.8156WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00037: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 157ms/step - loss: 0.5144 - accuracy: 0.8155 - val_loss: 0.7340 - val_accuracy: 0.7638
Epoch 38/80
236/237 [============================>.] - ETA: 0s - loss: 0.5048 - accuracy: 0.8248WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00038: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 157ms/step - loss: 0.5057 - accuracy: 0.8243 - val_loss: 1.2394 - val_accuracy: 0.5858
Epoch 39/80
236/237 [============================>.] - ETA: 0s - loss: 0.4794 - accuracy: 0.8259WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00039: val_loss did not improve from 0.54915
237/237 [==============================] - 38s 159ms/step - loss: 0.4782 - accuracy: 0.8261 - val_loss: 0.9839 - val_accuracy: 0.6398
Epoch 40/80
236/237 [============================>.] - ETA: 0s - loss: 0.4851 - accuracy: 0.8312WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00040: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 158ms/step - loss: 0.4846 - accuracy: 0.8314 - val_loss: 1.6195 - val_accuracy: 0.5106
Epoch 41/80
236/237 [============================>.] - ETA: 0s - loss: 0.4948 - accuracy: 0.8286WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00041: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 158ms/step - loss: 0.4954 - accuracy: 0.8285 - val_loss: 2.0649 - val_accuracy: 0.4640
Epoch 42/80
236/237 [============================>.] - ETA: 0s - loss: 0.4743 - accuracy: 0.8317WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00042: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 158ms/step - loss: 0.4739 - accuracy: 0.8319 - val_loss: 1.3076 - val_accuracy: 0.6144
Epoch 43/80
236/237 [============================>.] - ETA: 0s - loss: 0.4719 - accuracy: 0.8315WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00043: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 157ms/step - loss: 0.4717 - accuracy: 0.8314 - val_loss: 1.0668 - val_accuracy: 0.6472
Epoch 44/80
236/237 [============================>.] - ETA: 0s - loss: 0.4558 - accuracy: 0.8328WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00044: val_loss did not improve from 0.54915
237/237 [==============================] - 37s 156ms/step - loss: 0.4553 - accuracy: 0.8327 - val_loss: 0.7273 - val_accuracy: 0.7638
Epoch 45/80
236/237 [============================>.] - ETA: 0s - loss: 0.4625 - accuracy: 0.8289WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00045: val_loss did not improve from 0.54915
237/237 [==============================] - 38s 162ms/step - loss: 0.4646 - accuracy: 0.8281 - val_loss: 0.7154 - val_accuracy: 0.7500
Epoch 46/80
236/237 [============================>.] - ETA: 0s - loss: 0.4707 - accuracy: 0.8327WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00046: val_loss improved from 0.54915 to 0.50913, saving model to ./model_2.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.4702 - accuracy: 0.8326 - val_loss: 0.5091 - val_accuracy: 0.8019
Epoch 47/80
236/237 [============================>.] - ETA: 0s - loss: 0.4626 - accuracy: 0.8381WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00047: val_loss did not improve from 0.50913
237/237 [==============================] - 37s 158ms/step - loss: 0.4636 - accuracy: 0.8377 - val_loss: 0.7990 - val_accuracy: 0.7214
Epoch 48/80
236/237 [============================>.] - ETA: 0s - loss: 0.4401 - accuracy: 0.8307WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00048: val_loss improved from 0.50913 to 0.47999, saving model to ./model_2.h5
237/237 [==============================] - 37s 158ms/step - loss: 0.4397 - accuracy: 0.8309 - val_loss: 0.4800 - val_accuracy: 0.8273
Epoch 49/80
236/237 [============================>.] - ETA: 0s - loss: 0.4318 - accuracy: 0.8471WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00049: val_loss did not improve from 0.47999
237/237 [==============================] - 37s 155ms/step - loss: 0.4308 - accuracy: 0.8475 - val_loss: 0.8313 - val_accuracy: 0.7002
Epoch 50/80
236/237 [============================>.] - ETA: 0s - loss: 0.4424 - accuracy: 0.8366WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00050: val_loss did not improve from 0.47999
237/237 [==============================] - 38s 162ms/step - loss: 0.4416 - accuracy: 0.8368 - val_loss: 0.8426 - val_accuracy: 0.7066
Epoch 51/80
236/237 [============================>.] - ETA: 0s - loss: 0.4343 - accuracy: 0.8455WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00051: val_loss did not improve from 0.47999
237/237 [==============================] - 36s 153ms/step - loss: 0.4350 - accuracy: 0.8454 - val_loss: 0.6764 - val_accuracy: 0.7574
Epoch 52/80
236/237 [============================>.] - ETA: 0s - loss: 0.4209 - accuracy: 0.8474WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00052: val_loss did not improve from 0.47999
237/237 [==============================] - 38s 159ms/step - loss: 0.4210 - accuracy: 0.8470 - val_loss: 1.6707 - val_accuracy: 0.5127
Epoch 53/80
236/237 [============================>.] - ETA: 0s - loss: 0.4443 - accuracy: 0.8386WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00053: val_loss did not improve from 0.47999
237/237 [==============================] - 38s 160ms/step - loss: 0.4438 - accuracy: 0.8387 - val_loss: 0.5654 - val_accuracy: 0.8008
Epoch 54/80
236/237 [============================>.] - ETA: 0s - loss: 0.4206 - accuracy: 0.8450WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00054: val_loss improved from 0.47999 to 0.41329, saving model to ./model_2.h5
237/237 [==============================] - 38s 160ms/step - loss: 0.4198 - accuracy: 0.8454 - val_loss: 0.4133 - val_accuracy: 0.8453
Epoch 55/80
236/237 [============================>.] - ETA: 0s - loss: 0.4166 - accuracy: 0.8434WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00055: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 158ms/step - loss: 0.4166 - accuracy: 0.8433 - val_loss: 0.6809 - val_accuracy: 0.7532
Epoch 56/80
236/237 [============================>.] - ETA: 0s - loss: 0.4147 - accuracy: 0.8556WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00056: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 158ms/step - loss: 0.4141 - accuracy: 0.8557 - val_loss: 0.8106 - val_accuracy: 0.7256
Epoch 57/80
236/237 [============================>.] - ETA: 0s - loss: 0.4145 - accuracy: 0.8479WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00057: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 157ms/step - loss: 0.4137 - accuracy: 0.8483 - val_loss: 0.4913 - val_accuracy: 0.8146
Epoch 58/80
236/237 [============================>.] - ETA: 0s - loss: 0.4263 - accuracy: 0.8437WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00058: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 159ms/step - loss: 0.4264 - accuracy: 0.8438 - val_loss: 0.5002 - val_accuracy: 0.8252
Epoch 59/80
236/237 [============================>.] - ETA: 0s - loss: 0.4078 - accuracy: 0.8469WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00059: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 154ms/step - loss: 0.4074 - accuracy: 0.8470 - val_loss: 0.4821 - val_accuracy: 0.8178
Epoch 60/80
236/237 [============================>.] - ETA: 0s - loss: 0.3909 - accuracy: 0.8633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00060: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 161ms/step - loss: 0.3923 - accuracy: 0.8628 - val_loss: 0.4599 - val_accuracy: 0.8326
Epoch 61/80
236/237 [============================>.] - ETA: 0s - loss: 0.3951 - accuracy: 0.8612WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00061: val_loss did not improve from 0.41329
237/237 [==============================] - 36s 154ms/step - loss: 0.3940 - accuracy: 0.8615 - val_loss: 0.6475 - val_accuracy: 0.7733
Epoch 62/80
236/237 [============================>.] - ETA: 0s - loss: 0.3905 - accuracy: 0.8594WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00062: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 161ms/step - loss: 0.3902 - accuracy: 0.8592 - val_loss: 0.5071 - val_accuracy: 0.8178
Epoch 63/80
236/237 [============================>.] - ETA: 0s - loss: 0.3982 - accuracy: 0.8513WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00063: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 155ms/step - loss: 0.3971 - accuracy: 0.8517 - val_loss: 1.9427 - val_accuracy: 0.4788
Epoch 64/80
236/237 [============================>.] - ETA: 0s - loss: 0.3694 - accuracy: 0.8684WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00064: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 159ms/step - loss: 0.3686 - accuracy: 0.8684 - val_loss: 0.4324 - val_accuracy: 0.8411
Epoch 65/80
236/237 [============================>.] - ETA: 0s - loss: 0.3669 - accuracy: 0.8628WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00065: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 159ms/step - loss: 0.3702 - accuracy: 0.8618 - val_loss: 0.7636 - val_accuracy: 0.7352
Epoch 66/80
236/237 [============================>.] - ETA: 0s - loss: 0.3604 - accuracy: 0.8686WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00066: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 155ms/step - loss: 0.3601 - accuracy: 0.8689 - val_loss: 0.8498 - val_accuracy: 0.7034
Epoch 67/80
236/237 [============================>.] - ETA: 0s - loss: 0.3880 - accuracy: 0.8593WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00067: val_loss did not improve from 0.41329
237/237 [==============================] - 38s 158ms/step - loss: 0.3885 - accuracy: 0.8594 - val_loss: 0.7841 - val_accuracy: 0.7246
Epoch 68/80
236/237 [============================>.] - ETA: 0s - loss: 0.3753 - accuracy: 0.8646WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00068: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 156ms/step - loss: 0.3760 - accuracy: 0.8641 - val_loss: 0.7709 - val_accuracy: 0.6981
Epoch 69/80
236/237 [============================>.] - ETA: 0s - loss: 0.3755 - accuracy: 0.8633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00069: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 158ms/step - loss: 0.3758 - accuracy: 0.8634 - val_loss: 0.5512 - val_accuracy: 0.7913
Epoch 70/80
236/237 [============================>.] - ETA: 0s - loss: 0.3861 - accuracy: 0.8562WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00070: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 157ms/step - loss: 0.3854 - accuracy: 0.8565 - val_loss: 0.9561 - val_accuracy: 0.6960
Epoch 71/80
236/237 [============================>.] - ETA: 0s - loss: 0.3836 - accuracy: 0.8625WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00071: val_loss did not improve from 0.41329
237/237 [==============================] - 37s 154ms/step - loss: 0.3836 - accuracy: 0.8626 - val_loss: 1.0658 - val_accuracy: 0.6748
Epoch 72/80
236/237 [============================>.] - ETA: 0s - loss: 0.3610 - accuracy: 0.8654WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00072: val_loss improved from 0.41329 to 0.40984, saving model to ./model_2.h5
237/237 [==============================] - 38s 161ms/step - loss: 0.3602 - accuracy: 0.8658 - val_loss: 0.4098 - val_accuracy: 0.8485
Epoch 73/80
236/237 [============================>.] - ETA: 0s - loss: 0.3761 - accuracy: 0.8577WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00073: val_loss did not improve from 0.40984
237/237 [==============================] - 38s 158ms/step - loss: 0.3776 - accuracy: 0.8573 - val_loss: 0.4212 - val_accuracy: 0.8337
Epoch 74/80
236/237 [============================>.] - ETA: 0s - loss: 0.3667 - accuracy: 0.8694WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00074: val_loss did not improve from 0.40984
237/237 [==============================] - 37s 156ms/step - loss: 0.3659 - accuracy: 0.8697 - val_loss: 0.7450 - val_accuracy: 0.7511
Epoch 75/80
236/237 [============================>.] - ETA: 0s - loss: 0.3675 - accuracy: 0.8628WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00075: val_loss did not improve from 0.40984
237/237 [==============================] - 38s 159ms/step - loss: 0.3674 - accuracy: 0.8628 - val_loss: 0.6074 - val_accuracy: 0.7786
Epoch 76/80
236/237 [============================>.] - ETA: 0s - loss: 0.3495 - accuracy: 0.8615WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00076: val_loss did not improve from 0.40984
237/237 [==============================] - 38s 158ms/step - loss: 0.3494 - accuracy: 0.8618 - val_loss: 0.5279 - val_accuracy: 0.8199
Epoch 77/80
236/237 [============================>.] - ETA: 0s - loss: 0.3406 - accuracy: 0.8745WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00077: val_loss did not improve from 0.40984
237/237 [==============================] - 37s 157ms/step - loss: 0.3405 - accuracy: 0.8742 - val_loss: 0.7070 - val_accuracy: 0.7299
Epoch 78/80
236/237 [============================>.] - ETA: 0s - loss: 0.3641 - accuracy: 0.8676WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00078: val_loss improved from 0.40984 to 0.38156, saving model to ./model_2.h5
237/237 [==============================] - 36s 153ms/step - loss: 0.3644 - accuracy: 0.8673 - val_loss: 0.3816 - val_accuracy: 0.8633
Epoch 79/80
236/237 [============================>.] - ETA: 0s - loss: 0.3439 - accuracy: 0.8710WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00079: val_loss did not improve from 0.38156
237/237 [==============================] - 37s 158ms/step - loss: 0.3454 - accuracy: 0.8702 - val_loss: 0.5966 - val_accuracy: 0.7945
Epoch 80/80
236/237 [============================>.] - ETA: 0s - loss: 0.3659 - accuracy: 0.8657WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00080: val_loss did not improve from 0.38156
237/237 [==============================] - 38s 160ms/step - loss: 0.3661 - accuracy: 0.8660 - val_loss: 0.6089 - val_accuracy: 0.7754
Accuarcy accuracy: 76.91%
***************************************************************
Iteration =  3
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_10 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
batch_normalization_13 (Batc (None, 4, 4, 128)         512       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 2, 128)         147584    
_________________________________________________________________
batch_normalization_14 (Batc (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_5 (Dense)              (None, 12)                3084      
=================================================================
Total params: 315,532
Trainable params: 314,700
Non-trainable params: 832
_________________________________________________________________
Epoch 1/80
236/237 [============================>.] - ETA: 0s - loss: 2.2668 - accuracy: 0.2465WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00001: val_loss improved from inf to 2.74224, saving model to ./model_3.h5
237/237 [==============================] - 38s 162ms/step - loss: 2.2651 - accuracy: 0.2468 - val_loss: 2.7422 - val_accuracy: 0.0519
Epoch 2/80
236/237 [============================>.] - ETA: 0s - loss: 1.7324 - accuracy: 0.4023WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00002: val_loss improved from 2.74224 to 2.54085, saving model to ./model_3.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.7336 - accuracy: 0.4022 - val_loss: 2.5408 - val_accuracy: 0.2193
Epoch 3/80
236/237 [============================>.] - ETA: 0s - loss: 1.5648 - accuracy: 0.4623WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00003: val_loss improved from 2.54085 to 1.92419, saving model to ./model_3.h5
237/237 [==============================] - 39s 162ms/step - loss: 1.5630 - accuracy: 0.4627 - val_loss: 1.9242 - val_accuracy: 0.3125
Epoch 4/80
236/237 [============================>.] - ETA: 0s - loss: 1.3698 - accuracy: 0.5255WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00004: val_loss improved from 1.92419 to 1.77605, saving model to ./model_3.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.3694 - accuracy: 0.5259 - val_loss: 1.7760 - val_accuracy: 0.4216
Epoch 5/80
236/237 [============================>.] - ETA: 0s - loss: 1.2931 - accuracy: 0.5499WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00005: val_loss did not improve from 1.77605
237/237 [==============================] - 38s 161ms/step - loss: 1.2916 - accuracy: 0.5502 - val_loss: 2.1664 - val_accuracy: 0.3400
Epoch 6/80
236/237 [============================>.] - ETA: 0s - loss: 1.2143 - accuracy: 0.5730WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00006: val_loss did not improve from 1.77605
237/237 [==============================] - 38s 161ms/step - loss: 1.2127 - accuracy: 0.5735 - val_loss: 1.8129 - val_accuracy: 0.3983
Epoch 7/80
236/237 [============================>.] - ETA: 0s - loss: 1.1234 - accuracy: 0.6099WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00007: val_loss improved from 1.77605 to 1.24935, saving model to ./model_3.h5
237/237 [==============================] - 38s 160ms/step - loss: 1.1221 - accuracy: 0.6105 - val_loss: 1.2494 - val_accuracy: 0.5667
Epoch 8/80
236/237 [============================>.] - ETA: 0s - loss: 1.0729 - accuracy: 0.6287WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00008: val_loss did not improve from 1.24935
237/237 [==============================] - 38s 160ms/step - loss: 1.0721 - accuracy: 0.6284 - val_loss: 2.6512 - val_accuracy: 0.2807
Epoch 9/80
236/237 [============================>.] - ETA: 0s - loss: 0.9854 - accuracy: 0.6497WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00009: val_loss did not improve from 1.24935
237/237 [==============================] - 38s 160ms/step - loss: 0.9867 - accuracy: 0.6493 - val_loss: 1.3162 - val_accuracy: 0.5879
Epoch 10/80
236/237 [============================>.] - ETA: 0s - loss: 0.9514 - accuracy: 0.6699WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00010: val_loss did not improve from 1.24935
237/237 [==============================] - 38s 161ms/step - loss: 0.9529 - accuracy: 0.6702 - val_loss: 1.9314 - val_accuracy: 0.3655
Epoch 11/80
236/237 [============================>.] - ETA: 0s - loss: 0.9026 - accuracy: 0.6746WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00011: val_loss improved from 1.24935 to 1.11470, saving model to ./model_3.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.9043 - accuracy: 0.6739 - val_loss: 1.1147 - val_accuracy: 0.6102
Epoch 12/80
236/237 [============================>.] - ETA: 0s - loss: 0.8540 - accuracy: 0.6982WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00012: val_loss improved from 1.11470 to 0.89891, saving model to ./model_3.h5
237/237 [==============================] - 38s 161ms/step - loss: 0.8540 - accuracy: 0.6987 - val_loss: 0.8989 - val_accuracy: 0.6695
Epoch 13/80
236/237 [============================>.] - ETA: 0s - loss: 0.8119 - accuracy: 0.7131WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00013: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 156ms/step - loss: 0.8123 - accuracy: 0.7130 - val_loss: 1.3349 - val_accuracy: 0.5328
Epoch 14/80
236/237 [============================>.] - ETA: 0s - loss: 0.8232 - accuracy: 0.7102WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00014: val_loss did not improve from 0.89891
237/237 [==============================] - 38s 159ms/step - loss: 0.8227 - accuracy: 0.7109 - val_loss: 0.9681 - val_accuracy: 0.6568
Epoch 15/80
236/237 [============================>.] - ETA: 0s - loss: 0.7765 - accuracy: 0.7235WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00015: val_loss did not improve from 0.89891
237/237 [==============================] - 38s 159ms/step - loss: 0.7769 - accuracy: 0.7228 - val_loss: 1.4496 - val_accuracy: 0.4862
Epoch 16/80
236/237 [============================>.] - ETA: 0s - loss: 0.7842 - accuracy: 0.7190WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00016: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 158ms/step - loss: 0.7825 - accuracy: 0.7197 - val_loss: 1.6366 - val_accuracy: 0.4672
Epoch 17/80
236/237 [============================>.] - ETA: 0s - loss: 0.7521 - accuracy: 0.7444WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00017: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 158ms/step - loss: 0.7503 - accuracy: 0.7452 - val_loss: 1.8888 - val_accuracy: 0.4184
Epoch 18/80
236/237 [============================>.] - ETA: 0s - loss: 0.7206 - accuracy: 0.7487WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00018: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 156ms/step - loss: 0.7219 - accuracy: 0.7479 - val_loss: 1.6771 - val_accuracy: 0.4492
Epoch 19/80
236/237 [============================>.] - ETA: 0s - loss: 0.6981 - accuracy: 0.7495WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00019: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 157ms/step - loss: 0.6979 - accuracy: 0.7497 - val_loss: 1.2418 - val_accuracy: 0.5975
Epoch 20/80
236/237 [============================>.] - ETA: 0s - loss: 0.6989 - accuracy: 0.7497WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00020: val_loss did not improve from 0.89891
237/237 [==============================] - 38s 159ms/step - loss: 0.6996 - accuracy: 0.7497 - val_loss: 1.1534 - val_accuracy: 0.5657
Epoch 21/80
236/237 [============================>.] - ETA: 0s - loss: 0.6695 - accuracy: 0.7633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00021: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 156ms/step - loss: 0.6694 - accuracy: 0.7635 - val_loss: 1.3345 - val_accuracy: 0.5138
Epoch 22/80
236/237 [============================>.] - ETA: 0s - loss: 0.6356 - accuracy: 0.7755WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00022: val_loss did not improve from 0.89891
237/237 [==============================] - 38s 159ms/step - loss: 0.6347 - accuracy: 0.7756 - val_loss: 0.9468 - val_accuracy: 0.6727
Epoch 23/80
236/237 [============================>.] - ETA: 0s - loss: 0.6334 - accuracy: 0.7728WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00023: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 156ms/step - loss: 0.6340 - accuracy: 0.7727 - val_loss: 1.3960 - val_accuracy: 0.5487
Epoch 24/80
236/237 [============================>.] - ETA: 0s - loss: 0.6426 - accuracy: 0.7794WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00024: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 158ms/step - loss: 0.6422 - accuracy: 0.7793 - val_loss: 1.1824 - val_accuracy: 0.5794
Epoch 25/80
236/237 [============================>.] - ETA: 0s - loss: 0.6232 - accuracy: 0.7750WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00025: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 157ms/step - loss: 0.6229 - accuracy: 0.7752 - val_loss: 1.1179 - val_accuracy: 0.5900
Epoch 26/80
236/237 [============================>.] - ETA: 0s - loss: 0.6121 - accuracy: 0.7805WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00026: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 158ms/step - loss: 0.6107 - accuracy: 0.7809 - val_loss: 1.9304 - val_accuracy: 0.4110
Epoch 27/80
236/237 [============================>.] - ETA: 0s - loss: 0.5741 - accuracy: 0.7956WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00027: val_loss did not improve from 0.89891
237/237 [==============================] - 38s 159ms/step - loss: 0.5742 - accuracy: 0.7954 - val_loss: 1.1845 - val_accuracy: 0.6049
Epoch 28/80
236/237 [============================>.] - ETA: 0s - loss: 0.6015 - accuracy: 0.7795WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00028: val_loss did not improve from 0.89891
237/237 [==============================] - 37s 157ms/step - loss: 0.6005 - accuracy: 0.7802 - val_loss: 1.3441 - val_accuracy: 0.5625
Epoch 29/80
236/237 [============================>.] - ETA: 0s - loss: 0.5809 - accuracy: 0.7946WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00029: val_loss improved from 0.89891 to 0.64721, saving model to ./model_3.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.5820 - accuracy: 0.7936 - val_loss: 0.6472 - val_accuracy: 0.7617
Epoch 30/80
236/237 [============================>.] - ETA: 0s - loss: 0.5612 - accuracy: 0.7962WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00030: val_loss did not improve from 0.64721
237/237 [==============================] - 38s 159ms/step - loss: 0.5642 - accuracy: 0.7957 - val_loss: 1.1025 - val_accuracy: 0.6292
Epoch 31/80
236/237 [============================>.] - ETA: 0s - loss: 0.5529 - accuracy: 0.7972WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00031: val_loss did not improve from 0.64721
237/237 [==============================] - 36s 154ms/step - loss: 0.5523 - accuracy: 0.7973 - val_loss: 1.0754 - val_accuracy: 0.6176
Epoch 32/80
236/237 [============================>.] - ETA: 0s - loss: 0.5285 - accuracy: 0.8113WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00032: val_loss did not improve from 0.64721
237/237 [==============================] - 38s 159ms/step - loss: 0.5296 - accuracy: 0.8113 - val_loss: 1.0232 - val_accuracy: 0.6314
Epoch 33/80
236/237 [============================>.] - ETA: 0s - loss: 0.5187 - accuracy: 0.8230WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00033: val_loss did not improve from 0.64721
237/237 [==============================] - 37s 157ms/step - loss: 0.5181 - accuracy: 0.8232 - val_loss: 1.0279 - val_accuracy: 0.6462
Epoch 34/80
236/237 [============================>.] - ETA: 0s - loss: 0.5294 - accuracy: 0.8071WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00034: val_loss improved from 0.64721 to 0.56214, saving model to ./model_3.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.5309 - accuracy: 0.8066 - val_loss: 0.5621 - val_accuracy: 0.7998
Epoch 35/80
236/237 [============================>.] - ETA: 0s - loss: 0.5145 - accuracy: 0.8150WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00035: val_loss did not improve from 0.56214
237/237 [==============================] - 37s 158ms/step - loss: 0.5135 - accuracy: 0.8153 - val_loss: 1.0235 - val_accuracy: 0.6706
Epoch 36/80
236/237 [============================>.] - ETA: 0s - loss: 0.5279 - accuracy: 0.8166WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00036: val_loss did not improve from 0.56214
237/237 [==============================] - 38s 159ms/step - loss: 0.5282 - accuracy: 0.8161 - val_loss: 1.1982 - val_accuracy: 0.6165
Epoch 37/80
236/237 [============================>.] - ETA: 0s - loss: 0.4870 - accuracy: 0.8243WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00037: val_loss did not improve from 0.56214
237/237 [==============================] - 37s 154ms/step - loss: 0.4875 - accuracy: 0.8243 - val_loss: 1.8348 - val_accuracy: 0.4142
Epoch 38/80
236/237 [============================>.] - ETA: 0s - loss: 0.5086 - accuracy: 0.8190WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00038: val_loss did not improve from 0.56214
237/237 [==============================] - 38s 159ms/step - loss: 0.5076 - accuracy: 0.8195 - val_loss: 0.7867 - val_accuracy: 0.7150
Epoch 39/80
236/237 [============================>.] - ETA: 0s - loss: 0.4816 - accuracy: 0.8278WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00039: val_loss did not improve from 0.56214
237/237 [==============================] - 37s 156ms/step - loss: 0.4803 - accuracy: 0.8282 - val_loss: 0.9077 - val_accuracy: 0.6970
Epoch 40/80
236/237 [============================>.] - ETA: 0s - loss: 0.4998 - accuracy: 0.8150WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00040: val_loss did not improve from 0.56214
237/237 [==============================] - 37s 156ms/step - loss: 0.5003 - accuracy: 0.8150 - val_loss: 1.1114 - val_accuracy: 0.6261
Epoch 41/80
236/237 [============================>.] - ETA: 0s - loss: 0.4803 - accuracy: 0.8333WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00041: val_loss did not improve from 0.56214
237/237 [==============================] - 38s 160ms/step - loss: 0.4796 - accuracy: 0.8338 - val_loss: 0.6635 - val_accuracy: 0.7638
Epoch 42/80
236/237 [============================>.] - ETA: 0s - loss: 0.4878 - accuracy: 0.8275WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00042: val_loss improved from 0.56214 to 0.53332, saving model to ./model_3.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.4866 - accuracy: 0.8280 - val_loss: 0.5333 - val_accuracy: 0.8030
Epoch 43/80
236/237 [============================>.] - ETA: 0s - loss: 0.4610 - accuracy: 0.8374WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00043: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.4611 - accuracy: 0.8373 - val_loss: 0.5927 - val_accuracy: 0.7966
Epoch 44/80
236/237 [============================>.] - ETA: 0s - loss: 0.4761 - accuracy: 0.8255WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00044: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 160ms/step - loss: 0.4753 - accuracy: 0.8260 - val_loss: 0.8834 - val_accuracy: 0.7097
Epoch 45/80
236/237 [============================>.] - ETA: 0s - loss: 0.4658 - accuracy: 0.8336WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00045: val_loss did not improve from 0.53332
237/237 [==============================] - 36s 153ms/step - loss: 0.4658 - accuracy: 0.8335 - val_loss: 0.6475 - val_accuracy: 0.7701
Epoch 46/80
236/237 [============================>.] - ETA: 0s - loss: 0.4473 - accuracy: 0.8402WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00046: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 161ms/step - loss: 0.4470 - accuracy: 0.8401 - val_loss: 0.7892 - val_accuracy: 0.7267
Epoch 47/80
236/237 [============================>.] - ETA: 0s - loss: 0.4687 - accuracy: 0.8347WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00047: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.4687 - accuracy: 0.8343 - val_loss: 0.7724 - val_accuracy: 0.7161
Epoch 48/80
236/237 [============================>.] - ETA: 0s - loss: 0.4628 - accuracy: 0.8344WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00048: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 154ms/step - loss: 0.4632 - accuracy: 0.8343 - val_loss: 0.7637 - val_accuracy: 0.7172
Epoch 49/80
236/237 [============================>.] - ETA: 0s - loss: 0.4425 - accuracy: 0.8395WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00049: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 161ms/step - loss: 0.4417 - accuracy: 0.8397 - val_loss: 1.2263 - val_accuracy: 0.5964
Epoch 50/80
236/237 [============================>.] - ETA: 0s - loss: 0.4543 - accuracy: 0.8400WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00050: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.4559 - accuracy: 0.8399 - val_loss: 0.9135 - val_accuracy: 0.6949
Epoch 51/80
236/237 [============================>.] - ETA: 0s - loss: 0.4348 - accuracy: 0.8394WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00051: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.4352 - accuracy: 0.8390 - val_loss: 0.5828 - val_accuracy: 0.7828
Epoch 52/80
236/237 [============================>.] - ETA: 0s - loss: 0.4237 - accuracy: 0.8461WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00052: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 160ms/step - loss: 0.4234 - accuracy: 0.8462 - val_loss: 1.1517 - val_accuracy: 0.6229
Epoch 53/80
236/237 [============================>.] - ETA: 0s - loss: 0.4406 - accuracy: 0.8430WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00053: val_loss did not improve from 0.53332
237/237 [==============================] - 36s 154ms/step - loss: 0.4399 - accuracy: 0.8434 - val_loss: 0.8337 - val_accuracy: 0.7161
Epoch 54/80
236/237 [============================>.] - ETA: 0s - loss: 0.4228 - accuracy: 0.8511WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00054: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.4223 - accuracy: 0.8512 - val_loss: 0.6517 - val_accuracy: 0.7627
Epoch 55/80
236/237 [============================>.] - ETA: 0s - loss: 0.4358 - accuracy: 0.8410WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00055: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.4357 - accuracy: 0.8412 - val_loss: 1.2993 - val_accuracy: 0.5752
Epoch 56/80
236/237 [============================>.] - ETA: 0s - loss: 0.3920 - accuracy: 0.8596WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00056: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 160ms/step - loss: 0.3918 - accuracy: 0.8597 - val_loss: 0.6299 - val_accuracy: 0.7585
Epoch 57/80
236/237 [============================>.] - ETA: 0s - loss: 0.3994 - accuracy: 0.8508WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00057: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 156ms/step - loss: 0.4016 - accuracy: 0.8502 - val_loss: 0.6069 - val_accuracy: 0.7956
Epoch 58/80
236/237 [============================>.] - ETA: 0s - loss: 0.4182 - accuracy: 0.8453WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00058: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 160ms/step - loss: 0.4183 - accuracy: 0.8451 - val_loss: 0.8491 - val_accuracy: 0.6917
Epoch 59/80
236/237 [============================>.] - ETA: 0s - loss: 0.4093 - accuracy: 0.8472WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00059: val_loss did not improve from 0.53332
237/237 [==============================] - 36s 152ms/step - loss: 0.4106 - accuracy: 0.8473 - val_loss: 1.6663 - val_accuracy: 0.5148
Epoch 60/80
236/237 [============================>.] - ETA: 0s - loss: 0.4056 - accuracy: 0.8529WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00060: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.4048 - accuracy: 0.8533 - val_loss: 0.7427 - val_accuracy: 0.7299
Epoch 61/80
236/237 [============================>.] - ETA: 0s - loss: 0.4039 - accuracy: 0.8572WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00061: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 161ms/step - loss: 0.4035 - accuracy: 0.8570 - val_loss: 1.0857 - val_accuracy: 0.6419
Epoch 62/80
236/237 [============================>.] - ETA: 0s - loss: 0.4016 - accuracy: 0.8604WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00062: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 158ms/step - loss: 0.4009 - accuracy: 0.8610 - val_loss: 0.9281 - val_accuracy: 0.7087
Epoch 63/80
236/237 [============================>.] - ETA: 0s - loss: 0.4106 - accuracy: 0.8559WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00063: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 155ms/step - loss: 0.4097 - accuracy: 0.8562 - val_loss: 1.2162 - val_accuracy: 0.5900
Epoch 64/80
236/237 [============================>.] - ETA: 0s - loss: 0.3952 - accuracy: 0.8519WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00064: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 160ms/step - loss: 0.3947 - accuracy: 0.8520 - val_loss: 0.7939 - val_accuracy: 0.7172
Epoch 65/80
236/237 [============================>.] - ETA: 0s - loss: 0.3640 - accuracy: 0.8676WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00065: val_loss did not improve from 0.53332
237/237 [==============================] - 38s 159ms/step - loss: 0.3683 - accuracy: 0.8665 - val_loss: 0.6313 - val_accuracy: 0.7839
Epoch 66/80
236/237 [============================>.] - ETA: 0s - loss: 0.3887 - accuracy: 0.8581WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00066: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.3876 - accuracy: 0.8586 - val_loss: 0.5553 - val_accuracy: 0.8178
Epoch 67/80
236/237 [============================>.] - ETA: 0s - loss: 0.4001 - accuracy: 0.8564WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00067: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.3988 - accuracy: 0.8570 - val_loss: 0.9294 - val_accuracy: 0.7341
Epoch 68/80
236/237 [============================>.] - ETA: 0s - loss: 0.3972 - accuracy: 0.8493WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00068: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.3963 - accuracy: 0.8496 - val_loss: 0.6050 - val_accuracy: 0.7945
Epoch 69/80
236/237 [============================>.] - ETA: 0s - loss: 0.3809 - accuracy: 0.8620WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00069: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 156ms/step - loss: 0.3801 - accuracy: 0.8621 - val_loss: 0.6795 - val_accuracy: 0.7606
Epoch 70/80
236/237 [============================>.] - ETA: 0s - loss: 0.3811 - accuracy: 0.8572WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00070: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 158ms/step - loss: 0.3802 - accuracy: 0.8573 - val_loss: 0.6216 - val_accuracy: 0.7659
Epoch 71/80
236/237 [============================>.] - ETA: 0s - loss: 0.3626 - accuracy: 0.8705WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00071: val_loss did not improve from 0.53332
237/237 [==============================] - 37s 157ms/step - loss: 0.3620 - accuracy: 0.8705 - val_loss: 1.1191 - val_accuracy: 0.6811
Epoch 72/80
236/237 [============================>.] - ETA: 0s - loss: 0.4022 - accuracy: 0.8528WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00072: val_loss improved from 0.53332 to 0.41397, saving model to ./model_3.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.4038 - accuracy: 0.8523 - val_loss: 0.4140 - val_accuracy: 0.8475
Epoch 73/80
236/237 [============================>.] - ETA: 0s - loss: 0.3591 - accuracy: 0.8684WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00073: val_loss did not improve from 0.41397
237/237 [==============================] - 38s 159ms/step - loss: 0.3588 - accuracy: 0.8681 - val_loss: 1.0559 - val_accuracy: 0.6631
Epoch 74/80
236/237 [============================>.] - ETA: 0s - loss: 0.3856 - accuracy: 0.8578WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00074: val_loss did not improve from 0.41397
237/237 [==============================] - 37s 154ms/step - loss: 0.3857 - accuracy: 0.8571 - val_loss: 0.6130 - val_accuracy: 0.8104
Epoch 75/80
236/237 [============================>.] - ETA: 0s - loss: 0.3611 - accuracy: 0.8697WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00075: val_loss did not improve from 0.41397
237/237 [==============================] - 38s 159ms/step - loss: 0.3627 - accuracy: 0.8692 - val_loss: 0.8578 - val_accuracy: 0.7278
Epoch 76/80
236/237 [============================>.] - ETA: 0s - loss: 0.3612 - accuracy: 0.8639WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00076: val_loss improved from 0.41397 to 0.36873, saving model to ./model_3.h5
237/237 [==============================] - 37s 155ms/step - loss: 0.3604 - accuracy: 0.8642 - val_loss: 0.3687 - val_accuracy: 0.8665
Epoch 77/80
236/237 [============================>.] - ETA: 0s - loss: 0.3454 - accuracy: 0.8724WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00077: val_loss did not improve from 0.36873
237/237 [==============================] - 37s 156ms/step - loss: 0.3451 - accuracy: 0.8726 - val_loss: 0.3807 - val_accuracy: 0.8559
Epoch 78/80
236/237 [============================>.] - ETA: 0s - loss: 0.3694 - accuracy: 0.8652WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00078: val_loss did not improve from 0.36873
237/237 [==============================] - 38s 160ms/step - loss: 0.3702 - accuracy: 0.8647 - val_loss: 0.4569 - val_accuracy: 0.8337
Epoch 79/80
236/237 [============================>.] - ETA: 0s - loss: 0.3414 - accuracy: 0.8782WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00079: val_loss did not improve from 0.36873
237/237 [==============================] - 38s 161ms/step - loss: 0.3405 - accuracy: 0.8784 - val_loss: 0.5601 - val_accuracy: 0.7998
Epoch 80/80
236/237 [============================>.] - ETA: 0s - loss: 0.3746 - accuracy: 0.8599WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00080: val_loss did not improve from 0.36873
237/237 [==============================] - 37s 156ms/step - loss: 0.3738 - accuracy: 0.8599 - val_loss: 0.5138 - val_accuracy: 0.8189
Accuarcy accuracy: 82.63%
***************************************************************
Iteration =  4
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_15 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_15 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_16 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_17 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
batch_normalization_18 (Batc (None, 4, 4, 128)         512       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 2, 2, 128)         147584    
_________________________________________________________________
batch_normalization_19 (Batc (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 12)                3084      
=================================================================
Total params: 315,532
Trainable params: 314,700
Non-trainable params: 832
_________________________________________________________________
Epoch 1/80
236/237 [============================>.] - ETA: 0s - loss: 2.2230 - accuracy: 0.2495WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00001: val_loss improved from inf to 2.94509, saving model to ./model_4.h5
237/237 [==============================] - 39s 163ms/step - loss: 2.2208 - accuracy: 0.2500 - val_loss: 2.9451 - val_accuracy: 0.0381
Epoch 2/80
236/237 [============================>.] - ETA: 0s - loss: 1.7174 - accuracy: 0.4061WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00002: val_loss did not improve from 2.94509
237/237 [==============================] - 38s 162ms/step - loss: 1.7197 - accuracy: 0.4059 - val_loss: 3.6022 - val_accuracy: 0.1282
Epoch 3/80
236/237 [============================>.] - ETA: 0s - loss: 1.5299 - accuracy: 0.4732WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00003: val_loss improved from 2.94509 to 2.62618, saving model to ./model_4.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.5299 - accuracy: 0.4733 - val_loss: 2.6262 - val_accuracy: 0.1727
Epoch 4/80
236/237 [============================>.] - ETA: 0s - loss: 1.3870 - accuracy: 0.5194WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00004: val_loss improved from 2.62618 to 2.19620, saving model to ./model_4.h5
237/237 [==============================] - 39s 163ms/step - loss: 1.3846 - accuracy: 0.5206 - val_loss: 2.1962 - val_accuracy: 0.3792
Epoch 5/80
236/237 [============================>.] - ETA: 0s - loss: 1.2679 - accuracy: 0.5549WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00005: val_loss improved from 2.19620 to 1.55362, saving model to ./model_4.h5
237/237 [==============================] - 38s 161ms/step - loss: 1.2684 - accuracy: 0.5544 - val_loss: 1.5536 - val_accuracy: 0.4703
Epoch 6/80
236/237 [============================>.] - ETA: 0s - loss: 1.2057 - accuracy: 0.5825WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00006: val_loss did not improve from 1.55362
237/237 [==============================] - 38s 161ms/step - loss: 1.2063 - accuracy: 0.5827 - val_loss: 1.6876 - val_accuracy: 0.3941
Epoch 7/80
236/237 [============================>.] - ETA: 0s - loss: 1.1157 - accuracy: 0.6189WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00007: val_loss did not improve from 1.55362
237/237 [==============================] - 38s 161ms/step - loss: 1.1142 - accuracy: 0.6195 - val_loss: 1.9040 - val_accuracy: 0.4343
Epoch 8/80
236/237 [============================>.] - ETA: 0s - loss: 1.0480 - accuracy: 0.6399WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00008: val_loss improved from 1.55362 to 1.31697, saving model to ./model_4.h5
237/237 [==============================] - 38s 161ms/step - loss: 1.0492 - accuracy: 0.6390 - val_loss: 1.3170 - val_accuracy: 0.5583
Epoch 9/80
236/237 [============================>.] - ETA: 0s - loss: 1.0002 - accuracy: 0.6472WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00009: val_loss improved from 1.31697 to 1.08702, saving model to ./model_4.h5
237/237 [==============================] - 38s 161ms/step - loss: 1.0001 - accuracy: 0.6477 - val_loss: 1.0870 - val_accuracy: 0.6409
Epoch 10/80
236/237 [============================>.] - ETA: 0s - loss: 0.9262 - accuracy: 0.6947WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00010: val_loss did not improve from 1.08702
237/237 [==============================] - 38s 159ms/step - loss: 0.9253 - accuracy: 0.6949 - val_loss: 1.5499 - val_accuracy: 0.5085
Epoch 11/80
236/237 [============================>.] - ETA: 0s - loss: 0.9084 - accuracy: 0.6807WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00011: val_loss improved from 1.08702 to 0.85461, saving model to ./model_4.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.9076 - accuracy: 0.6805 - val_loss: 0.8546 - val_accuracy: 0.7034
Epoch 12/80
236/237 [============================>.] - ETA: 0s - loss: 0.8514 - accuracy: 0.6982WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00012: val_loss did not improve from 0.85461
237/237 [==============================] - 38s 159ms/step - loss: 0.8497 - accuracy: 0.6990 - val_loss: 1.8428 - val_accuracy: 0.4089
Epoch 13/80
236/237 [============================>.] - ETA: 0s - loss: 0.8208 - accuracy: 0.7237WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00013: val_loss did not improve from 0.85461
237/237 [==============================] - 38s 159ms/step - loss: 0.8208 - accuracy: 0.7241 - val_loss: 1.6044 - val_accuracy: 0.4926
Epoch 14/80
236/237 [============================>.] - ETA: 0s - loss: 0.7741 - accuracy: 0.7264WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00014: val_loss did not improve from 0.85461
237/237 [==============================] - 38s 159ms/step - loss: 0.7744 - accuracy: 0.7270 - val_loss: 1.0560 - val_accuracy: 0.6314
Epoch 15/80
236/237 [============================>.] - ETA: 0s - loss: 0.7730 - accuracy: 0.7229WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00015: val_loss did not improve from 0.85461
237/237 [==============================] - 37s 157ms/step - loss: 0.7734 - accuracy: 0.7230 - val_loss: 1.6898 - val_accuracy: 0.4566
Epoch 16/80
236/237 [============================>.] - ETA: 0s - loss: 0.7507 - accuracy: 0.7325WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00016: val_loss did not improve from 0.85461
237/237 [==============================] - 38s 159ms/step - loss: 0.7510 - accuracy: 0.7320 - val_loss: 1.3783 - val_accuracy: 0.5318
Epoch 17/80
236/237 [============================>.] - ETA: 0s - loss: 0.7250 - accuracy: 0.7447WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00017: val_loss did not improve from 0.85461
237/237 [==============================] - 37s 157ms/step - loss: 0.7250 - accuracy: 0.7437 - val_loss: 1.0265 - val_accuracy: 0.6419
Epoch 18/80
236/237 [============================>.] - ETA: 0s - loss: 0.7194 - accuracy: 0.7465WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00018: val_loss did not improve from 0.85461
237/237 [==============================] - 37s 158ms/step - loss: 0.7196 - accuracy: 0.7468 - val_loss: 1.4109 - val_accuracy: 0.5456
Epoch 19/80
236/237 [============================>.] - ETA: 0s - loss: 0.7049 - accuracy: 0.7487WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00019: val_loss improved from 0.85461 to 0.69037, saving model to ./model_4.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.7050 - accuracy: 0.7487 - val_loss: 0.6904 - val_accuracy: 0.7479
Epoch 20/80
236/237 [============================>.] - ETA: 0s - loss: 0.6689 - accuracy: 0.7635WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00020: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 157ms/step - loss: 0.6685 - accuracy: 0.7640 - val_loss: 0.7020 - val_accuracy: 0.7521
Epoch 21/80
236/237 [============================>.] - ETA: 0s - loss: 0.6481 - accuracy: 0.7710WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00021: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 158ms/step - loss: 0.6474 - accuracy: 0.7709 - val_loss: 1.1174 - val_accuracy: 0.6303
Epoch 22/80
236/237 [============================>.] - ETA: 0s - loss: 0.6521 - accuracy: 0.7665WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00022: val_loss did not improve from 0.69037
237/237 [==============================] - 38s 159ms/step - loss: 0.6511 - accuracy: 0.7664 - val_loss: 1.4962 - val_accuracy: 0.5286
Epoch 23/80
236/237 [============================>.] - ETA: 0s - loss: 0.6190 - accuracy: 0.7776WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00023: val_loss did not improve from 0.69037
237/237 [==============================] - 38s 160ms/step - loss: 0.6201 - accuracy: 0.7770 - val_loss: 1.3303 - val_accuracy: 0.5614
Epoch 24/80
236/237 [============================>.] - ETA: 0s - loss: 0.6056 - accuracy: 0.7818WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00024: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 157ms/step - loss: 0.6052 - accuracy: 0.7817 - val_loss: 1.5271 - val_accuracy: 0.5254
Epoch 25/80
236/237 [============================>.] - ETA: 0s - loss: 0.6164 - accuracy: 0.7845WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00025: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 158ms/step - loss: 0.6158 - accuracy: 0.7846 - val_loss: 1.6767 - val_accuracy: 0.4936
Epoch 26/80
236/237 [============================>.] - ETA: 0s - loss: 0.5986 - accuracy: 0.7890WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00026: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 157ms/step - loss: 0.5993 - accuracy: 0.7888 - val_loss: 0.8547 - val_accuracy: 0.7140
Epoch 27/80
236/237 [============================>.] - ETA: 0s - loss: 0.5748 - accuracy: 0.7970WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00027: val_loss did not improve from 0.69037
237/237 [==============================] - 38s 159ms/step - loss: 0.5763 - accuracy: 0.7962 - val_loss: 0.7308 - val_accuracy: 0.7331
Epoch 28/80
236/237 [============================>.] - ETA: 0s - loss: 0.5835 - accuracy: 0.7895WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00028: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 157ms/step - loss: 0.5837 - accuracy: 0.7891 - val_loss: 1.1086 - val_accuracy: 0.6430
Epoch 29/80
236/237 [============================>.] - ETA: 0s - loss: 0.5578 - accuracy: 0.7893WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00029: val_loss did not improve from 0.69037
237/237 [==============================] - 37s 154ms/step - loss: 0.5572 - accuracy: 0.7894 - val_loss: 1.1760 - val_accuracy: 0.6345
Epoch 30/80
236/237 [============================>.] - ETA: 0s - loss: 0.5775 - accuracy: 0.7930WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00030: val_loss improved from 0.69037 to 0.62027, saving model to ./model_4.h5
237/237 [==============================] - 38s 160ms/step - loss: 0.5769 - accuracy: 0.7933 - val_loss: 0.6203 - val_accuracy: 0.7966
Epoch 31/80
236/237 [============================>.] - ETA: 0s - loss: 0.5380 - accuracy: 0.8079WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00031: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 157ms/step - loss: 0.5369 - accuracy: 0.8081 - val_loss: 2.3259 - val_accuracy: 0.4375
Epoch 32/80
236/237 [============================>.] - ETA: 0s - loss: 0.5548 - accuracy: 0.8020WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00032: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 157ms/step - loss: 0.5542 - accuracy: 0.8021 - val_loss: 2.2287 - val_accuracy: 0.4089
Epoch 33/80
236/237 [============================>.] - ETA: 0s - loss: 0.5240 - accuracy: 0.8100WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00033: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 157ms/step - loss: 0.5237 - accuracy: 0.8103 - val_loss: 1.7723 - val_accuracy: 0.5275
Epoch 34/80
236/237 [============================>.] - ETA: 0s - loss: 0.5236 - accuracy: 0.8091WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00034: val_loss did not improve from 0.62027
237/237 [==============================] - 38s 160ms/step - loss: 0.5233 - accuracy: 0.8093 - val_loss: 0.7353 - val_accuracy: 0.7246
Epoch 35/80
236/237 [============================>.] - ETA: 0s - loss: 0.5225 - accuracy: 0.8120WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00035: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 155ms/step - loss: 0.5221 - accuracy: 0.8122 - val_loss: 2.5522 - val_accuracy: 0.3475
Epoch 36/80
236/237 [============================>.] - ETA: 0s - loss: 0.5313 - accuracy: 0.8057WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00036: val_loss did not improve from 0.62027
237/237 [==============================] - 38s 159ms/step - loss: 0.5308 - accuracy: 0.8058 - val_loss: 0.7452 - val_accuracy: 0.7500
Epoch 37/80
236/237 [============================>.] - ETA: 0s - loss: 0.4820 - accuracy: 0.8336WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00037: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 155ms/step - loss: 0.4807 - accuracy: 0.8343 - val_loss: 2.8177 - val_accuracy: 0.3602
Epoch 38/80
236/237 [============================>.] - ETA: 0s - loss: 0.5036 - accuracy: 0.8129WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00038: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 157ms/step - loss: 0.5039 - accuracy: 0.8129 - val_loss: 1.8046 - val_accuracy: 0.4746
Epoch 39/80
236/237 [============================>.] - ETA: 0s - loss: 0.4799 - accuracy: 0.8296WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00039: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 158ms/step - loss: 0.4812 - accuracy: 0.8293 - val_loss: 0.7219 - val_accuracy: 0.7595
Epoch 40/80
236/237 [============================>.] - ETA: 0s - loss: 0.4971 - accuracy: 0.8209WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00040: val_loss did not improve from 0.62027
237/237 [==============================] - 38s 159ms/step - loss: 0.4980 - accuracy: 0.8208 - val_loss: 0.7365 - val_accuracy: 0.7405
Epoch 41/80
236/237 [============================>.] - ETA: 0s - loss: 0.5050 - accuracy: 0.8230WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00041: val_loss did not improve from 0.62027
237/237 [==============================] - 37s 157ms/step - loss: 0.5040 - accuracy: 0.8235 - val_loss: 0.7889 - val_accuracy: 0.7341
Epoch 42/80
236/237 [============================>.] - ETA: 0s - loss: 0.4800 - accuracy: 0.8234WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00042: val_loss improved from 0.62027 to 0.50256, saving model to ./model_4.h5
237/237 [==============================] - 38s 160ms/step - loss: 0.4797 - accuracy: 0.8233 - val_loss: 0.5026 - val_accuracy: 0.8252
Epoch 43/80
236/237 [============================>.] - ETA: 0s - loss: 0.4676 - accuracy: 0.8351WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00043: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.4687 - accuracy: 0.8353 - val_loss: 1.9797 - val_accuracy: 0.4862
Epoch 44/80
236/237 [============================>.] - ETA: 0s - loss: 0.4770 - accuracy: 0.8272WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00044: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 156ms/step - loss: 0.4760 - accuracy: 0.8280 - val_loss: 0.5804 - val_accuracy: 0.7966
Epoch 45/80
236/237 [============================>.] - ETA: 0s - loss: 0.4723 - accuracy: 0.8280WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00045: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 155ms/step - loss: 0.4733 - accuracy: 0.8277 - val_loss: 1.4219 - val_accuracy: 0.5794
Epoch 46/80
236/237 [============================>.] - ETA: 0s - loss: 0.4515 - accuracy: 0.8307WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00046: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 160ms/step - loss: 0.4524 - accuracy: 0.8309 - val_loss: 1.0603 - val_accuracy: 0.6780
Epoch 47/80
236/237 [============================>.] - ETA: 0s - loss: 0.4549 - accuracy: 0.8355WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00047: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 154ms/step - loss: 0.4558 - accuracy: 0.8354 - val_loss: 0.6058 - val_accuracy: 0.7913
Epoch 48/80
236/237 [============================>.] - ETA: 0s - loss: 0.4417 - accuracy: 0.8408WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00048: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.4412 - accuracy: 0.8409 - val_loss: 0.8978 - val_accuracy: 0.7203
Epoch 49/80
236/237 [============================>.] - ETA: 0s - loss: 0.4156 - accuracy: 0.8520WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00049: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 161ms/step - loss: 0.4170 - accuracy: 0.8513 - val_loss: 0.7187 - val_accuracy: 0.7638
Epoch 50/80
236/237 [============================>.] - ETA: 0s - loss: 0.4702 - accuracy: 0.8301WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00050: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 157ms/step - loss: 0.4698 - accuracy: 0.8302 - val_loss: 1.1281 - val_accuracy: 0.6525
Epoch 51/80
236/237 [============================>.] - ETA: 0s - loss: 0.4410 - accuracy: 0.8458WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00051: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.4400 - accuracy: 0.8462 - val_loss: 1.3330 - val_accuracy: 0.6049
Epoch 52/80
236/237 [============================>.] - ETA: 0s - loss: 0.4131 - accuracy: 0.8448WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00052: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 161ms/step - loss: 0.4121 - accuracy: 0.8452 - val_loss: 1.9431 - val_accuracy: 0.4725
Epoch 53/80
236/237 [============================>.] - ETA: 0s - loss: 0.4322 - accuracy: 0.8402WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00053: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 155ms/step - loss: 0.4314 - accuracy: 0.8406 - val_loss: 0.8565 - val_accuracy: 0.7087
Epoch 54/80
236/237 [============================>.] - ETA: 0s - loss: 0.4365 - accuracy: 0.8400WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00054: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 157ms/step - loss: 0.4354 - accuracy: 0.8404 - val_loss: 0.5221 - val_accuracy: 0.8146
Epoch 55/80
236/237 [============================>.] - ETA: 0s - loss: 0.3915 - accuracy: 0.8633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00055: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 160ms/step - loss: 0.3917 - accuracy: 0.8631 - val_loss: 2.7031 - val_accuracy: 0.3983
Epoch 56/80
236/237 [============================>.] - ETA: 0s - loss: 0.4197 - accuracy: 0.8490WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00056: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 155ms/step - loss: 0.4193 - accuracy: 0.8491 - val_loss: 0.5433 - val_accuracy: 0.8083
Epoch 57/80
236/237 [============================>.] - ETA: 0s - loss: 0.4153 - accuracy: 0.8511WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00057: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.4157 - accuracy: 0.8512 - val_loss: 0.9937 - val_accuracy: 0.6769
Epoch 58/80
236/237 [============================>.] - ETA: 0s - loss: 0.4237 - accuracy: 0.8447WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00058: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 159ms/step - loss: 0.4234 - accuracy: 0.8451 - val_loss: 0.5326 - val_accuracy: 0.8051
Epoch 59/80
236/237 [============================>.] - ETA: 0s - loss: 0.4120 - accuracy: 0.8469WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00059: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 159ms/step - loss: 0.4109 - accuracy: 0.8470 - val_loss: 0.6384 - val_accuracy: 0.7956
Epoch 60/80
236/237 [============================>.] - ETA: 0s - loss: 0.4561 - accuracy: 0.8408WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00060: val_loss did not improve from 0.50256
237/237 [==============================] - 38s 159ms/step - loss: 0.4558 - accuracy: 0.8409 - val_loss: 0.7076 - val_accuracy: 0.7564
Epoch 61/80
236/237 [============================>.] - ETA: 0s - loss: 0.3972 - accuracy: 0.8585WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00061: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 155ms/step - loss: 0.3985 - accuracy: 0.8584 - val_loss: 1.3084 - val_accuracy: 0.6208
Epoch 62/80
236/237 [============================>.] - ETA: 0s - loss: 0.4008 - accuracy: 0.8439WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00062: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 157ms/step - loss: 0.4024 - accuracy: 0.8436 - val_loss: 0.7464 - val_accuracy: 0.7479
Epoch 63/80
236/237 [============================>.] - ETA: 0s - loss: 0.3938 - accuracy: 0.8556WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00063: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.3928 - accuracy: 0.8557 - val_loss: 0.8147 - val_accuracy: 0.7267
Epoch 64/80
236/237 [============================>.] - ETA: 0s - loss: 0.4025 - accuracy: 0.8490WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00064: val_loss did not improve from 0.50256
237/237 [==============================] - 37s 158ms/step - loss: 0.4017 - accuracy: 0.8494 - val_loss: 1.0491 - val_accuracy: 0.6737
Epoch 65/80
236/237 [============================>.] - ETA: 0s - loss: 0.3620 - accuracy: 0.8593WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00065: val_loss improved from 0.50256 to 0.36767, saving model to ./model_4.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.3623 - accuracy: 0.8594 - val_loss: 0.3677 - val_accuracy: 0.8729
Epoch 66/80
236/237 [============================>.] - ETA: 0s - loss: 0.3949 - accuracy: 0.8578WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00066: val_loss did not improve from 0.36767
237/237 [==============================] - 38s 159ms/step - loss: 0.3958 - accuracy: 0.8568 - val_loss: 1.5732 - val_accuracy: 0.5816
Epoch 67/80
236/237 [============================>.] - ETA: 0s - loss: 0.3717 - accuracy: 0.8617WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00067: val_loss did not improve from 0.36767
237/237 [==============================] - 38s 158ms/step - loss: 0.3725 - accuracy: 0.8618 - val_loss: 1.3724 - val_accuracy: 0.5900
Epoch 68/80
236/237 [============================>.] - ETA: 0s - loss: 0.4055 - accuracy: 0.8495WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00068: val_loss did not improve from 0.36767
237/237 [==============================] - 37s 157ms/step - loss: 0.4062 - accuracy: 0.8491 - val_loss: 0.3689 - val_accuracy: 0.8792
Epoch 69/80
236/237 [============================>.] - ETA: 0s - loss: 0.3709 - accuracy: 0.8612WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00069: val_loss did not improve from 0.36767
237/237 [==============================] - 37s 158ms/step - loss: 0.3713 - accuracy: 0.8610 - val_loss: 0.3884 - val_accuracy: 0.8602
Epoch 70/80
236/237 [============================>.] - ETA: 0s - loss: 0.3611 - accuracy: 0.8681WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00070: val_loss did not improve from 0.36767
237/237 [==============================] - 37s 158ms/step - loss: 0.3620 - accuracy: 0.8678 - val_loss: 0.6581 - val_accuracy: 0.7871
Epoch 71/80
236/237 [============================>.] - ETA: 0s - loss: 0.3773 - accuracy: 0.8633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00071: val_loss did not improve from 0.36767
237/237 [==============================] - 38s 162ms/step - loss: 0.3776 - accuracy: 0.8634 - val_loss: 0.8562 - val_accuracy: 0.7055
Epoch 72/80
236/237 [============================>.] - ETA: 0s - loss: 0.3817 - accuracy: 0.8585WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00072: val_loss did not improve from 0.36767
237/237 [==============================] - 36s 151ms/step - loss: 0.3830 - accuracy: 0.8581 - val_loss: 0.8319 - val_accuracy: 0.7278
Epoch 73/80
236/237 [============================>.] - ETA: 0s - loss: 0.3856 - accuracy: 0.8567WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00073: val_loss did not improve from 0.36767
237/237 [==============================] - 37s 155ms/step - loss: 0.3848 - accuracy: 0.8571 - val_loss: 0.6840 - val_accuracy: 0.7775
Epoch 74/80
236/237 [============================>.] - ETA: 0s - loss: 0.3613 - accuracy: 0.8652WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00074: val_loss did not improve from 0.36767
237/237 [==============================] - 38s 160ms/step - loss: 0.3616 - accuracy: 0.8644 - val_loss: 0.7929 - val_accuracy: 0.7341
Epoch 75/80
236/237 [============================>.] - ETA: 0s - loss: 0.3736 - accuracy: 0.8660WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00075: val_loss did not improve from 0.36767
237/237 [==============================] - 38s 160ms/step - loss: 0.3748 - accuracy: 0.8652 - val_loss: 0.4352 - val_accuracy: 0.8633
Epoch 76/80
236/237 [============================>.] - ETA: 0s - loss: 0.3971 - accuracy: 0.8535WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00076: val_loss improved from 0.36767 to 0.36372, saving model to ./model_4.h5
237/237 [==============================] - 37s 155ms/step - loss: 0.3975 - accuracy: 0.8533 - val_loss: 0.3637 - val_accuracy: 0.8803
Epoch 77/80
236/237 [============================>.] - ETA: 0s - loss: 0.3501 - accuracy: 0.8700WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00077: val_loss did not improve from 0.36372
237/237 [==============================] - 38s 159ms/step - loss: 0.3500 - accuracy: 0.8702 - val_loss: 0.3872 - val_accuracy: 0.8655
Epoch 78/80
236/237 [============================>.] - ETA: 0s - loss: 0.3747 - accuracy: 0.8596WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00078: val_loss did not improve from 0.36372
237/237 [==============================] - 37s 157ms/step - loss: 0.3750 - accuracy: 0.8594 - val_loss: 0.6083 - val_accuracy: 0.7722
Epoch 79/80
236/237 [============================>.] - ETA: 0s - loss: 0.3366 - accuracy: 0.8734WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00079: val_loss did not improve from 0.36372
237/237 [==============================] - 38s 160ms/step - loss: 0.3375 - accuracy: 0.8734 - val_loss: 0.3696 - val_accuracy: 0.8686
Epoch 80/80
236/237 [============================>.] - ETA: 0s - loss: 0.3846 - accuracy: 0.8588WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00080: val_loss did not improve from 0.36372
237/237 [==============================] - 38s 159ms/step - loss: 0.3841 - accuracy: 0.8588 - val_loss: 1.5541 - val_accuracy: 0.5614
Accuarcy accuracy: 56.99%
***************************************************************
Iteration =  5
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_20 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_20 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_21 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_22 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
batch_normalization_23 (Batc (None, 4, 4, 128)         512       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 2, 2, 128)         147584    
_________________________________________________________________
batch_normalization_24 (Batc (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_9 (Dense)              (None, 12)                3084      
=================================================================
Total params: 315,532
Trainable params: 314,700
Non-trainable params: 832
_________________________________________________________________
Epoch 1/80
236/237 [============================>.] - ETA: 0s - loss: 2.1741 - accuracy: 0.2673WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00001: val_loss improved from inf to 2.72035, saving model to ./model_5.h5
237/237 [==============================] - 39s 163ms/step - loss: 2.1728 - accuracy: 0.2674 - val_loss: 2.7204 - val_accuracy: 0.1547
Epoch 2/80
236/237 [============================>.] - ETA: 0s - loss: 1.7051 - accuracy: 0.4121WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00002: val_loss improved from 2.72035 to 2.60804, saving model to ./model_5.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.7040 - accuracy: 0.4123 - val_loss: 2.6080 - val_accuracy: 0.2489
Epoch 3/80
236/237 [============================>.] - ETA: 0s - loss: 1.5323 - accuracy: 0.4639WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00003: val_loss improved from 2.60804 to 1.94135, saving model to ./model_5.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.5307 - accuracy: 0.4651 - val_loss: 1.9414 - val_accuracy: 0.3316
Epoch 4/80
236/237 [============================>.] - ETA: 0s - loss: 1.3617 - accuracy: 0.5199WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00004: val_loss did not improve from 1.94135
237/237 [==============================] - 38s 161ms/step - loss: 1.3634 - accuracy: 0.5198 - val_loss: 2.3155 - val_accuracy: 0.2892
Epoch 5/80
236/237 [============================>.] - ETA: 0s - loss: 1.2491 - accuracy: 0.5727WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00005: val_loss did not improve from 1.94135
237/237 [==============================] - 38s 161ms/step - loss: 1.2480 - accuracy: 0.5732 - val_loss: 2.1053 - val_accuracy: 0.3210
Epoch 6/80
236/237 [============================>.] - ETA: 0s - loss: 1.1799 - accuracy: 0.5998WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00006: val_loss improved from 1.94135 to 1.51440, saving model to ./model_5.h5
237/237 [==============================] - 38s 162ms/step - loss: 1.1791 - accuracy: 0.5999 - val_loss: 1.5144 - val_accuracy: 0.4661
Epoch 7/80
236/237 [============================>.] - ETA: 0s - loss: 1.0900 - accuracy: 0.6202WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00007: val_loss improved from 1.51440 to 1.22478, saving model to ./model_5.h5
237/237 [==============================] - 38s 161ms/step - loss: 1.0914 - accuracy: 0.6202 - val_loss: 1.2248 - val_accuracy: 0.6006
Epoch 8/80
236/237 [============================>.] - ETA: 0s - loss: 1.0129 - accuracy: 0.6521WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00008: val_loss improved from 1.22478 to 0.94356, saving model to ./model_5.h5
237/237 [==============================] - 38s 160ms/step - loss: 1.0142 - accuracy: 0.6520 - val_loss: 0.9436 - val_accuracy: 0.6843
Epoch 9/80
236/237 [============================>.] - ETA: 0s - loss: 0.9868 - accuracy: 0.6553WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00009: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 160ms/step - loss: 0.9890 - accuracy: 0.6546 - val_loss: 1.8597 - val_accuracy: 0.4322
Epoch 10/80
236/237 [============================>.] - ETA: 0s - loss: 0.9534 - accuracy: 0.6706WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00010: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 160ms/step - loss: 0.9519 - accuracy: 0.6715 - val_loss: 2.7430 - val_accuracy: 0.2818
Epoch 11/80
236/237 [============================>.] - ETA: 0s - loss: 0.8990 - accuracy: 0.6807WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00011: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 159ms/step - loss: 0.8985 - accuracy: 0.6805 - val_loss: 1.7092 - val_accuracy: 0.4258
Epoch 12/80
236/237 [============================>.] - ETA: 0s - loss: 0.8603 - accuracy: 0.6953WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00012: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 159ms/step - loss: 0.8580 - accuracy: 0.6964 - val_loss: 1.2337 - val_accuracy: 0.5710
Epoch 13/80
236/237 [============================>.] - ETA: 0s - loss: 0.8217 - accuracy: 0.7147WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00013: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 160ms/step - loss: 0.8211 - accuracy: 0.7149 - val_loss: 1.7015 - val_accuracy: 0.4725
Epoch 14/80
236/237 [============================>.] - ETA: 0s - loss: 0.8157 - accuracy: 0.7205WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00014: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 158ms/step - loss: 0.8163 - accuracy: 0.7201 - val_loss: 1.7806 - val_accuracy: 0.4841
Epoch 15/80
236/237 [============================>.] - ETA: 0s - loss: 0.8217 - accuracy: 0.7168WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00015: val_loss did not improve from 0.94356
237/237 [==============================] - 38s 159ms/step - loss: 0.8245 - accuracy: 0.7159 - val_loss: 1.1864 - val_accuracy: 0.5932
Epoch 16/80
236/237 [============================>.] - ETA: 0s - loss: 0.7741 - accuracy: 0.7357WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00016: val_loss improved from 0.94356 to 0.92874, saving model to ./model_5.h5
237/237 [==============================] - 37s 156ms/step - loss: 0.7729 - accuracy: 0.7363 - val_loss: 0.9287 - val_accuracy: 0.6970
Epoch 17/80
236/237 [============================>.] - ETA: 0s - loss: 0.7199 - accuracy: 0.7407WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00017: val_loss did not improve from 0.92874
237/237 [==============================] - 38s 159ms/step - loss: 0.7200 - accuracy: 0.7405 - val_loss: 1.5329 - val_accuracy: 0.4873
Epoch 18/80
236/237 [============================>.] - ETA: 0s - loss: 0.7487 - accuracy: 0.7391WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00018: val_loss did not improve from 0.92874
237/237 [==============================] - 38s 159ms/step - loss: 0.7489 - accuracy: 0.7397 - val_loss: 1.0579 - val_accuracy: 0.6324
Epoch 19/80
236/237 [============================>.] - ETA: 0s - loss: 0.6984 - accuracy: 0.7545WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00019: val_loss did not improve from 0.92874
237/237 [==============================] - 37s 157ms/step - loss: 0.7010 - accuracy: 0.7537 - val_loss: 1.4217 - val_accuracy: 0.5540
Epoch 20/80
236/237 [============================>.] - ETA: 0s - loss: 0.6859 - accuracy: 0.7577WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00020: val_loss improved from 0.92874 to 0.74349, saving model to ./model_5.h5
237/237 [==============================] - 38s 159ms/step - loss: 0.6852 - accuracy: 0.7576 - val_loss: 0.7435 - val_accuracy: 0.7362
Epoch 21/80
236/237 [============================>.] - ETA: 0s - loss: 0.6347 - accuracy: 0.7790WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00021: val_loss did not improve from 0.74349
237/237 [==============================] - 37s 156ms/step - loss: 0.6374 - accuracy: 0.7783 - val_loss: 1.6287 - val_accuracy: 0.5138
Epoch 22/80
236/237 [============================>.] - ETA: 0s - loss: 0.6724 - accuracy: 0.7662WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00022: val_loss did not improve from 0.74349
237/237 [==============================] - 37s 156ms/step - loss: 0.6731 - accuracy: 0.7661 - val_loss: 1.0487 - val_accuracy: 0.6483
Epoch 23/80
236/237 [============================>.] - ETA: 0s - loss: 0.6482 - accuracy: 0.7757WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00023: val_loss did not improve from 0.74349
237/237 [==============================] - 38s 158ms/step - loss: 0.6486 - accuracy: 0.7756 - val_loss: 0.8754 - val_accuracy: 0.7214
Epoch 24/80
236/237 [============================>.] - ETA: 0s - loss: 0.6307 - accuracy: 0.7797WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00024: val_loss did not improve from 0.74349
237/237 [==============================] - 38s 158ms/step - loss: 0.6313 - accuracy: 0.7793 - val_loss: 0.9384 - val_accuracy: 0.6896
Epoch 25/80
236/237 [============================>.] - ETA: 0s - loss: 0.6311 - accuracy: 0.7745WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00025: val_loss did not improve from 0.74349
237/237 [==============================] - 37s 156ms/step - loss: 0.6338 - accuracy: 0.7738 - val_loss: 0.8345 - val_accuracy: 0.7288
Epoch 26/80
236/237 [============================>.] - ETA: 0s - loss: 0.5834 - accuracy: 0.7962WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00026: val_loss did not improve from 0.74349
237/237 [==============================] - 38s 159ms/step - loss: 0.5833 - accuracy: 0.7962 - val_loss: 2.2894 - val_accuracy: 0.3729
Epoch 27/80
236/237 [============================>.] - ETA: 0s - loss: 0.5751 - accuracy: 0.7930WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00027: val_loss did not improve from 0.74349
237/237 [==============================] - 37s 157ms/step - loss: 0.5742 - accuracy: 0.7933 - val_loss: 1.1475 - val_accuracy: 0.6155
Epoch 28/80
236/237 [============================>.] - ETA: 0s - loss: 0.5852 - accuracy: 0.7906WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00028: val_loss did not improve from 0.74349
237/237 [==============================] - 38s 160ms/step - loss: 0.5854 - accuracy: 0.7904 - val_loss: 1.2760 - val_accuracy: 0.5572
Epoch 29/80
236/237 [============================>.] - ETA: 0s - loss: 0.5598 - accuracy: 0.8004WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00029: val_loss improved from 0.74349 to 0.68478, saving model to ./model_5.h5
237/237 [==============================] - 37s 155ms/step - loss: 0.5588 - accuracy: 0.8010 - val_loss: 0.6848 - val_accuracy: 0.7500
Epoch 30/80
236/237 [============================>.] - ETA: 0s - loss: 0.5493 - accuracy: 0.8033WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00030: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 158ms/step - loss: 0.5492 - accuracy: 0.8036 - val_loss: 1.3619 - val_accuracy: 0.5604
Epoch 31/80
236/237 [============================>.] - ETA: 0s - loss: 0.5740 - accuracy: 0.7983WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00031: val_loss did not improve from 0.68478
237/237 [==============================] - 38s 158ms/step - loss: 0.5743 - accuracy: 0.7981 - val_loss: 0.7695 - val_accuracy: 0.7468
Epoch 32/80
236/237 [============================>.] - ETA: 0s - loss: 0.5419 - accuracy: 0.8088WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00032: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 155ms/step - loss: 0.5432 - accuracy: 0.8089 - val_loss: 0.9949 - val_accuracy: 0.6684
Epoch 33/80
236/237 [============================>.] - ETA: 0s - loss: 0.5484 - accuracy: 0.8084WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00033: val_loss did not improve from 0.68478
237/237 [==============================] - 38s 159ms/step - loss: 0.5482 - accuracy: 0.8084 - val_loss: 0.7502 - val_accuracy: 0.7415
Epoch 34/80
236/237 [============================>.] - ETA: 0s - loss: 0.5018 - accuracy: 0.8193WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00034: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 157ms/step - loss: 0.5014 - accuracy: 0.8195 - val_loss: 0.8470 - val_accuracy: 0.7267
Epoch 35/80
236/237 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.8081WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00035: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 156ms/step - loss: 0.5326 - accuracy: 0.8081 - val_loss: 0.7791 - val_accuracy: 0.7267
Epoch 36/80
236/237 [============================>.] - ETA: 0s - loss: 0.5287 - accuracy: 0.8015WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00036: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 158ms/step - loss: 0.5291 - accuracy: 0.8015 - val_loss: 1.1421 - val_accuracy: 0.6451
Epoch 37/80
236/237 [============================>.] - ETA: 0s - loss: 0.4868 - accuracy: 0.8227WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00037: val_loss did not improve from 0.68478
237/237 [==============================] - 37s 157ms/step - loss: 0.4883 - accuracy: 0.8221 - val_loss: 0.7536 - val_accuracy: 0.7299
Epoch 38/80
236/237 [============================>.] - ETA: 0s - loss: 0.5161 - accuracy: 0.8140WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00038: val_loss improved from 0.68478 to 0.67275, saving model to ./model_5.h5
237/237 [==============================] - 38s 158ms/step - loss: 0.5148 - accuracy: 0.8145 - val_loss: 0.6728 - val_accuracy: 0.7733
Epoch 39/80
236/237 [============================>.] - ETA: 0s - loss: 0.5020 - accuracy: 0.8163WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00039: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 155ms/step - loss: 0.5007 - accuracy: 0.8169 - val_loss: 1.3212 - val_accuracy: 0.5911
Epoch 40/80
236/237 [============================>.] - ETA: 0s - loss: 0.4935 - accuracy: 0.8207WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00040: val_loss did not improve from 0.67275
237/237 [==============================] - 38s 159ms/step - loss: 0.4941 - accuracy: 0.8207 - val_loss: 1.5256 - val_accuracy: 0.5667
Epoch 41/80
236/237 [============================>.] - ETA: 0s - loss: 0.4879 - accuracy: 0.8253WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00041: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 157ms/step - loss: 0.4877 - accuracy: 0.8252 - val_loss: 0.9236 - val_accuracy: 0.6864
Epoch 42/80
236/237 [============================>.] - ETA: 0s - loss: 0.4882 - accuracy: 0.8161WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00042: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 155ms/step - loss: 0.4877 - accuracy: 0.8161 - val_loss: 0.8947 - val_accuracy: 0.7097
Epoch 43/80
236/237 [============================>.] - ETA: 0s - loss: 0.4811 - accuracy: 0.8227WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00043: val_loss did not improve from 0.67275
237/237 [==============================] - 38s 159ms/step - loss: 0.4811 - accuracy: 0.8227 - val_loss: 0.8896 - val_accuracy: 0.6949
Epoch 44/80
236/237 [============================>.] - ETA: 0s - loss: 0.4908 - accuracy: 0.8246WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00044: val_loss did not improve from 0.67275
237/237 [==============================] - 38s 158ms/step - loss: 0.4905 - accuracy: 0.8248 - val_loss: 1.1276 - val_accuracy: 0.5922
Epoch 45/80
236/237 [============================>.] - ETA: 0s - loss: 0.4583 - accuracy: 0.8374WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00045: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 156ms/step - loss: 0.4584 - accuracy: 0.8370 - val_loss: 0.7698 - val_accuracy: 0.7362
Epoch 46/80
236/237 [============================>.] - ETA: 0s - loss: 0.4833 - accuracy: 0.8271WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00046: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 156ms/step - loss: 0.4860 - accuracy: 0.8265 - val_loss: 1.4839 - val_accuracy: 0.5117
Epoch 47/80
236/237 [============================>.] - ETA: 0s - loss: 0.4602 - accuracy: 0.8316WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00047: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 158ms/step - loss: 0.4592 - accuracy: 0.8320 - val_loss: 1.0914 - val_accuracy: 0.6271
Epoch 48/80
236/237 [============================>.] - ETA: 0s - loss: 0.4347 - accuracy: 0.8410WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00048: val_loss did not improve from 0.67275
237/237 [==============================] - 37s 156ms/step - loss: 0.4335 - accuracy: 0.8416 - val_loss: 0.7670 - val_accuracy: 0.7394
Epoch 49/80
236/237 [============================>.] - ETA: 0s - loss: 0.4543 - accuracy: 0.8315WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00049: val_loss improved from 0.67275 to 0.57358, saving model to ./model_5.h5
237/237 [==============================] - 38s 161ms/step - loss: 0.4546 - accuracy: 0.8311 - val_loss: 0.5736 - val_accuracy: 0.8284
Epoch 50/80
236/237 [============================>.] - ETA: 0s - loss: 0.4403 - accuracy: 0.8406WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00050: val_loss did not improve from 0.57358
237/237 [==============================] - 37s 156ms/step - loss: 0.4415 - accuracy: 0.8399 - val_loss: 0.5813 - val_accuracy: 0.8051
Epoch 51/80
236/237 [============================>.] - ETA: 0s - loss: 0.4471 - accuracy: 0.8444WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00051: val_loss did not improve from 0.57358
237/237 [==============================] - 38s 158ms/step - loss: 0.4467 - accuracy: 0.8443 - val_loss: 1.4741 - val_accuracy: 0.5530
Epoch 52/80
236/237 [============================>.] - ETA: 0s - loss: 0.4283 - accuracy: 0.8360WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00052: val_loss did not improve from 0.57358
237/237 [==============================] - 38s 158ms/step - loss: 0.4284 - accuracy: 0.8364 - val_loss: 0.6401 - val_accuracy: 0.7701
Epoch 53/80
236/237 [============================>.] - ETA: 0s - loss: 0.4386 - accuracy: 0.8429WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00053: val_loss did not improve from 0.57358
237/237 [==============================] - 36s 154ms/step - loss: 0.4379 - accuracy: 0.8430 - val_loss: 1.0292 - val_accuracy: 0.6949
Epoch 54/80
236/237 [============================>.] - ETA: 0s - loss: 0.4398 - accuracy: 0.8381WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00054: val_loss did not improve from 0.57358
237/237 [==============================] - 38s 159ms/step - loss: 0.4393 - accuracy: 0.8380 - val_loss: 3.0619 - val_accuracy: 0.4015
Epoch 55/80
236/237 [============================>.] - ETA: 0s - loss: 0.4173 - accuracy: 0.8450WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00055: val_loss did not improve from 0.57358
237/237 [==============================] - 37s 157ms/step - loss: 0.4165 - accuracy: 0.8454 - val_loss: 0.6752 - val_accuracy: 0.7733
Epoch 56/80
236/237 [============================>.] - ETA: 0s - loss: 0.4139 - accuracy: 0.8482WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00056: val_loss improved from 0.57358 to 0.50974, saving model to ./model_5.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.4145 - accuracy: 0.8480 - val_loss: 0.5097 - val_accuracy: 0.8369
Epoch 57/80
236/237 [============================>.] - ETA: 0s - loss: 0.4350 - accuracy: 0.8402WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00057: val_loss did not improve from 0.50974
237/237 [==============================] - 38s 159ms/step - loss: 0.4348 - accuracy: 0.8404 - val_loss: 1.1574 - val_accuracy: 0.6525
Epoch 58/80
236/237 [============================>.] - ETA: 0s - loss: 0.4003 - accuracy: 0.8508WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00058: val_loss did not improve from 0.50974
237/237 [==============================] - 37s 158ms/step - loss: 0.4011 - accuracy: 0.8510 - val_loss: 0.6195 - val_accuracy: 0.7892
Epoch 59/80
236/237 [============================>.] - ETA: 0s - loss: 0.4247 - accuracy: 0.8466WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00059: val_loss improved from 0.50974 to 0.45822, saving model to ./model_5.h5
237/237 [==============================] - 37s 155ms/step - loss: 0.4246 - accuracy: 0.8465 - val_loss: 0.4582 - val_accuracy: 0.8464
Epoch 60/80
236/237 [============================>.] - ETA: 0s - loss: 0.4153 - accuracy: 0.8556WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00060: val_loss did not improve from 0.45822
237/237 [==============================] - 37s 155ms/step - loss: 0.4161 - accuracy: 0.8554 - val_loss: 0.4806 - val_accuracy: 0.8326
Epoch 61/80
236/237 [============================>.] - ETA: 0s - loss: 0.4120 - accuracy: 0.8517WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00061: val_loss did not improve from 0.45822
237/237 [==============================] - 38s 161ms/step - loss: 0.4120 - accuracy: 0.8518 - val_loss: 0.4786 - val_accuracy: 0.8326
Epoch 62/80
236/237 [============================>.] - ETA: 0s - loss: 0.4131 - accuracy: 0.8521WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00062: val_loss did not improve from 0.45822
237/237 [==============================] - 37s 157ms/step - loss: 0.4124 - accuracy: 0.8522 - val_loss: 0.9082 - val_accuracy: 0.7044
Epoch 63/80
236/237 [============================>.] - ETA: 0s - loss: 0.3940 - accuracy: 0.8551WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00063: val_loss did not improve from 0.45822
237/237 [==============================] - 37s 157ms/step - loss: 0.3958 - accuracy: 0.8541 - val_loss: 1.0752 - val_accuracy: 0.6600
Epoch 64/80
236/237 [============================>.] - ETA: 0s - loss: 0.3805 - accuracy: 0.8570WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00064: val_loss did not improve from 0.45822
237/237 [==============================] - 38s 159ms/step - loss: 0.3794 - accuracy: 0.8573 - val_loss: 0.5840 - val_accuracy: 0.7977
Epoch 65/80
236/237 [============================>.] - ETA: 0s - loss: 0.3798 - accuracy: 0.8633WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00065: val_loss did not improve from 0.45822
237/237 [==============================] - 38s 159ms/step - loss: 0.3808 - accuracy: 0.8628 - val_loss: 0.7869 - val_accuracy: 0.7436
Epoch 66/80
236/237 [============================>.] - ETA: 0s - loss: 0.3878 - accuracy: 0.8601WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00066: val_loss improved from 0.45822 to 0.45267, saving model to ./model_5.h5
237/237 [==============================] - 36s 154ms/step - loss: 0.3874 - accuracy: 0.8602 - val_loss: 0.4527 - val_accuracy: 0.8422
Epoch 67/80
236/237 [============================>.] - ETA: 0s - loss: 0.3664 - accuracy: 0.8689WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00067: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 156ms/step - loss: 0.3671 - accuracy: 0.8687 - val_loss: 0.8661 - val_accuracy: 0.7320
Epoch 68/80
236/237 [============================>.] - ETA: 0s - loss: 0.3850 - accuracy: 0.8529WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00068: val_loss did not improve from 0.45267
237/237 [==============================] - 38s 161ms/step - loss: 0.3847 - accuracy: 0.8530 - val_loss: 0.5064 - val_accuracy: 0.8263
Epoch 69/80
236/237 [============================>.] - ETA: 0s - loss: 0.3936 - accuracy: 0.8580WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00069: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 155ms/step - loss: 0.3946 - accuracy: 0.8581 - val_loss: 0.5287 - val_accuracy: 0.8284
Epoch 70/80
236/237 [============================>.] - ETA: 0s - loss: 0.3752 - accuracy: 0.8609WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00070: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 157ms/step - loss: 0.3755 - accuracy: 0.8610 - val_loss: 0.4896 - val_accuracy: 0.8411
Epoch 71/80
236/237 [============================>.] - ETA: 0s - loss: 0.3735 - accuracy: 0.8668WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00071: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 157ms/step - loss: 0.3730 - accuracy: 0.8671 - val_loss: 0.5497 - val_accuracy: 0.8157
Epoch 72/80
236/237 [============================>.] - ETA: 0s - loss: 0.3925 - accuracy: 0.8594WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00072: val_loss did not improve from 0.45267
237/237 [==============================] - 39s 163ms/step - loss: 0.3920 - accuracy: 0.8592 - val_loss: 0.9225 - val_accuracy: 0.7193
Epoch 73/80
236/237 [============================>.] - ETA: 0s - loss: 0.3665 - accuracy: 0.8697WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00073: val_loss did not improve from 0.45267
237/237 [==============================] - 36s 152ms/step - loss: 0.3658 - accuracy: 0.8700 - val_loss: 0.5167 - val_accuracy: 0.8475
Epoch 74/80
236/237 [============================>.] - ETA: 0s - loss: 0.3731 - accuracy: 0.8575WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00074: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 157ms/step - loss: 0.3734 - accuracy: 0.8570 - val_loss: 0.6022 - val_accuracy: 0.8061
Epoch 75/80
236/237 [============================>.] - ETA: 0s - loss: 0.3861 - accuracy: 0.8567WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00075: val_loss did not improve from 0.45267
237/237 [==============================] - 38s 160ms/step - loss: 0.3867 - accuracy: 0.8568 - val_loss: 0.5060 - val_accuracy: 0.8316
Epoch 76/80
236/237 [============================>.] - ETA: 0s - loss: 0.3679 - accuracy: 0.8654WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00076: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 157ms/step - loss: 0.3676 - accuracy: 0.8652 - val_loss: 0.6024 - val_accuracy: 0.8273
Epoch 77/80
236/237 [============================>.] - ETA: 0s - loss: 0.3877 - accuracy: 0.8626WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00077: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 157ms/step - loss: 0.3873 - accuracy: 0.8626 - val_loss: 0.6464 - val_accuracy: 0.8083
Epoch 78/80
236/237 [============================>.] - ETA: 0s - loss: 0.3435 - accuracy: 0.8750WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00078: val_loss did not improve from 0.45267
237/237 [==============================] - 37s 158ms/step - loss: 0.3471 - accuracy: 0.8739 - val_loss: 0.8776 - val_accuracy: 0.7309
Epoch 79/80
236/237 [============================>.] - ETA: 0s - loss: 0.3611 - accuracy: 0.8729WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00079: val_loss did not improve from 0.45267
237/237 [==============================] - 38s 162ms/step - loss: 0.3609 - accuracy: 0.8729 - val_loss: 0.4754 - val_accuracy: 0.8316
Epoch 80/80
236/237 [============================>.] - ETA: 0s - loss: 0.3643 - accuracy: 0.8628WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

Epoch 00080: val_loss improved from 0.45267 to 0.44773, saving model to ./model_5.h5
237/237 [==============================] - 37s 157ms/step - loss: 0.3645 - accuracy: 0.8631 - val_loss: 0.4477 - val_accuracy: 0.8443
Accuarcy accuracy: 85.17%
***************************************************************
Training finished...
Accuracy: 75.44% (+/- 9.89%)
WARNING:tensorflow:From /home/sarubi/miniconda3/envs/cnn/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./best_model/assets

5-kold corss validation accuracy on different image shapes.

image_shape=(66,66,3): 
* 1 - Accuarcy accuracy: 75.53%
* 2 - Accuarcy accuracy: 76.91%
* 3 - Accuarcy accuracy: 82.63%
* 4 - Accuarcy accuracy: 56.99%
* 5 - Accuarcy accuracy: 85.17% 
* Avg. Accuracy: 75.44% (+/- 9.89%)

image_shape=(128,128,3):
* 1 - Accuarcy accuracy: 84.11%
* 2 - Accuarcy accuracy: 56.25%
* 3 - Accuarcy accuracy: 75.00%
* 4 - Accuarcy accuracy: 67.80%
* 5 - Accuarcy accuracy: 81.14% 
* Avg. Accuracy: 72.86% (+/- 10.01%) 


image_shape=(256,256,3): 
* 1 - Accuarcy accuracy: 72.56%
* 2 - Accuarcy accuracy: 35.06%
* 3 - Accuarcy accuracy: 86.44%
* 4 - Accuarcy accuracy: 79.77%
* 5 - Accuarcy accuracy: 35.17%
* Avg. Accuracy: 61.80% (+/- 22.23%)

image_shape=(370,370,3): PC machine
* 1 - Accuarcy accuracy: 60.70%
* 2 - Accuarcy accuracy: 65.47%
* 3 - Accuarcy accuracy: 60.17%
* 4 - 
* 5 -
* Avg.


Best accracy on image_shape=(66,66,3) with 5-fold corss validation:

1. Accuarcy accuracy: 75.53%
1. Accuarcy accuracy: 76.91%
1. Accuarcy accuracy: 82.63%
1. Accuarcy accuracy: 56.99%
1. Accuarcy accuracy: 85.17%

Average Accuracy: 75.44% (+/- 9.89%)

# Final Test Prediction

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Read all test data. 
test_data = []
for file in os.listdir(test_path):
    test_data.append([file])
final_test_data = pd.DataFrame(test_data, columns=['imagefile'])
final_test_data.shape

In [ ]:
class_indices={'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}
class_indices

In [ ]:
!ls /kaggle

In [ ]:
def plant_seeds_classifier():
    
    print("Start Predicting......")
    
    datagen_test = ImageDataGenerator(rescale=1./255)
    
    test_generator = datagen_test.flow_from_dataframe(dataframe=final_test_data,
                                                        directory=test_path,
                                                        x_col="imagefile",
                                                        y_col=None,
                                                        batch_size=1,
                                                        seed=SEED,
                                                        shuffle=False,
                                                        class_mode=None,
                                                        target_size=(66, 66),
                                                        color_mode='rgb');
        
    model = load_model('../input/bestmodel/best_model')
    filenames = test_generator.filenames
    nb_samples = len(filenames)
    print("Total test files: " +str(nb_samples) )

    predictions = model.predict_generator(test_generator,steps = nb_samples) # return prob of each class per image (softmax)
    
    predicted_class_indices=np.argmax(predictions,axis=1)
    print("Predicted_class_indices: " +str(len(predicted_class_indices)))
       
    labels = dict((v,k) for k,v in class_indices.items())
    predicted_labels = [labels[k] for k in predicted_class_indices]
    print("Predicted_labels: " +str(len(predicted_labels)))

    results=pd.DataFrame({"file":filenames,
                          "species":predicted_labels})

    print (results)
    print("results length: " + str(len(results)))
    
    results.to_csv("submission.csv",index=False)

    print("Prediction Completed..")

In [ ]:
plant_seeds_classifier()

# References


1. https://www.kaggle.com/c/plant-seedlings-classification/notebooks?competitionId=7880
1. https://www.kaggle.com/praanj/basic-keras-cnn-with-startified-kfold-evaluation
1. https://www.kaggle.com/kinsomaz/plant-seedling-classification-using-keras-0-9726
1. https://www.kaggle.com/kaiyungtan/plant-seedlings-classification
1. https://www.kaggle.com/akashwadhwaai/plant-seedlings-classification